In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import auc, roc_auc_score, roc_curve, confusion_matrix, classification_report

from keras.models import Model, load_model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
#from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from the correct location
#from keras.preprocessing.sequence import pad_sequences
#from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
datal = pd.DataFrame(pd.read_excel('/content/drive/MyDrive/camerounaise_spam_data.xlsx'))
#datal = datal = pd.read_csv('/content/drive/MyDrive/smsspamcollection/spam.csv',encoding='latin-1')

scale_mapped={
    'Spam':1.0,
    'spam':1.0,
    'ham':0.0,
    'h':0.0
}

datal.rename(columns = {'v1':'label','v2':'SMS'},inplace=True)
# Checking the rows and columns(m*n)
data = (datal[['label','SMS']].dropna().reset_index(drop=True)).sample(frac = 1)
print(len(data))
# Checking for the nulls
data.info()
# Re-Checking for nulls
data.isnull().sum()
# Row-wise all values are displayed
data.values

22553
<class 'pandas.core.frame.DataFrame'>
Index: 22553 entries, 12023 to 10947
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   22553 non-null  object
 1   SMS     22553 non-null  object
dtypes: object(2)
memory usage: 528.6+ KB


array([['ham', 'Et la ville de Ottawa ??'],
       ['ham', 'C le délégué qui dit sa?'],
       ['ham',
        'Hummmm ma sœur tu ris, yen à qui ont même parfois 40 de température sans avoir le palu ohh'],
       ...,
       ['ham', "S'il vous plaît on doit venir comme d'hab à 8h"],
       ['ham', 'You have to find a backup solution @23778776876'],
       ['ham',
        "Hummmm lorsqu'une personne est vraiment importante pour toi c'est normale"]],
      dtype=object)

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Libraries needed for NLP
import nltk
import re

# Stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# Stemmers
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# Lemmatizer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
!pip install fasttext

  Using cached fasttext-0.9.3.tar.gz (73 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313487 sha256=c12c6ed498c1a52b78627f7cd6085e38039c47b35d9361521a0e71cb5138afce
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [ ]:
import fasttext

PRETRAINED_MODEL_PATH = '/content/drive/MyDrive/lid.176.bin'
model_fastext = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [ ]:
data_sms = []
text = []
# Store it in a variable
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
# Cleanup
# Store it in a variable
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def clean_data_en(text):
  '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
  # replace email address with 'emailaddress'
  text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", r'emailadress ', str(text))
  # replace urls with 'webaddress'
  #text = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,6}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", r'webaddress', str(text))
  text = re.sub(r"(https?|ftp)://[^\s/$.?#].[^\s]*", r'webaddress', str(text))
  text = re.sub( r'\b(?:https?:\/\/)?(?:\*?\w+\*?\s*)?(bit\.ly|eianal\.com|onelink\.to|viaonelink\.to|cutt\.ly)\/[^\s]*\b', r'webaddress', str(text))
  # replace 10 digit phone number with 'phone-number'
  #text =  re.sub('(?:\d{8}(?:\d{2}(?:\d{2})?)?|\(\+?\d{2,3}\)\s?(?:\d{4}[\s*.-]?\d{4}|\d{3}[\s*.-]?\d{3}|\d{2}([\s*.-]?)\d{2}\1\d{2}(?:\1\d{2})?))', r'phone-number', text)
  text =  re.sub( r'(?:\+?237|00237|237)?[ -*./]*(6[235-9][ -*./]*[0-9]{2}[ -*./]*[0-9]{2}[ -*./]*[0-9]{2}|[1-9][ -*./]*[0-9]{2}[ -*./]*[0-9]{2}[ -*./]*[0-9]{2}[ -*./]*[0-9]{2})', r'phoneNumber', text)
  text = re.sub(r'\b\d[\d\s,\.]*\s*(FCFA|F|fr|fcf|f|fcfa|xfa|XFA|u|U)\b', r' money ', text)
  #text = re.sub('[\d]{1,10}fcfa|[\d]{1,10}FCFA|[\d]{1,10}Fr|[\d]{1,10}fr|[\d]{1,10}FCFA|[\d]{1,10} fcfa|[\d]{1,10} FCFA|[\d]{1,10} Fr|[\d]{1,10} fr', r' money ', text)
  # remove punctuation
  text=re.sub('[^\w\d\s]', r' ',text)
  # remove whitespace between terms with single space
  text = re.sub('\s+', r' ', text)
  # remove leading and trailing whitespace
  text = re.sub('^\s+|\s*?$', r' ',text)
  # change words to lower case
  text = re.sub('(https://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)', r' url ', text)
  text = re.sub(r'(?:\+?237|00237|237)? *(6[235-9] *[0-9]{7}|[1-9] *[0-9]{7})', r' phoneNumber ', text)
  text = re.sub('[\d]{1,10}GO|[\d]{1,10}Go|[\d]{1,10}gO|[\d]{1,10}go|[\d]{1,10} GO|[\d]{1,10} Go|[\d]{1,10} gO|[\d]{1,10} go', r' dataConnexion ', text)
  text = re.sub('([http//[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)| (https://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)|(www.https://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)|(www.http//[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)', r'webaddress', text)
  # replace normal number with 'number'
  text = re.sub('\d+(\.\d+)?', r'Nombre ', text)
  text = text.lower()
  return text

def clean_data_fr(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
      and remove words containing numbers.'''
    # replace email address with 'emailaddress'
    text = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", r'adresseemail ', str(text))
    # replace urls with 'webaddress'
    text = re.sub( r"(https?|ftp)://[^\s/$.?#].[^\s]*", r'adresseweb ', str(text))
    text = re.sub( r'\b(?:https?:\/\/)?(?:\*?\w+\*?\s*)?(bit\.ly|eianal\.com|onelink\.to|viaonelink\.to|cutt\.ly)\/[^\s]*\b', r'adresseweb ', str(text))
    # replace 10 digit phone number with 'phone-number'
    text =  re.sub(r'(?:\+?237|00237|237)?[ -*./]*(6[235-9][ -*./]*[0-9]{2}[ -*./]*[0-9]{2}[ -*./]*[0-9]{2}|[1-9][ -*./]*[0-9]{2}[ -*./]*[0-9]{2}[ -*./]*[0-9]{2}[ -*./]*[0-9]{2})', r'phoneNumber ', text)
    text = re.sub(r'\b\d[\d\s,\.]*\s*(FCFA|F|fr|fcf|f|fcfa|xfa|XFA|u|U)\b', r' argent ', text)
    #text = re.sub('[\d]{1,10}fcfa|[\d]{1,10}FCFA|[\d]{1,10}Fr|[\d]{1,10}fr|[\d]{1,10}FCFA|[\d]{1,10} fcfa|[\d]{1,10} FCFA|[\d]{1,10} Fr|[\d]{1,10} fr', r' argent ', text)
    # remove punctuation
    text=re.sub('[^\w\d\s]', r' ',text)
    # remove whitespace between terms with single space
    text = re.sub('\s+', r' ', text)
    # remove leading and trailing whitespace
    text = re.sub('^\s+|\s*?$', r' ',text)
    # change words to lower case
    text = re.sub('(https://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)', r' url ', text)
    text = re.sub('(00237[1-9] *[0-9]{8}|\+237[1-9] *[0-9]{8}|(237[1-9] *[0-9]{8})|(6[2|3|5-9] *[0-9]{7}))', r' numeroTelephone ', text)
    text = re.sub('[\d]{1,10}GO|[\d]{1,10}Go|[\d]{1,10}gO|[\d]{1,10}go|[\d]{1,10} GO|[\d]{1,10} Go|[\d]{1,10} gO|[\d]{1,10} go', r' donneeInternet ', text)
    text = re.sub('([http//[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)| (https://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)|(www.https://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)|(www.http//[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$)', r' adresseWeb ', text)
    # replace normal number with 'number'
    text = re.sub('\d+(\.\d+)?', r'Nombre ', text)
    text = text.lower()
    return text


In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Déclaration du stemmer et lemmatizer
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Chargement des stopwords
more_stopwords = ['u', 'im', 'c']
englishStopWord = stopwords.words("english") + more_stopwords
frenchStopWord = stopwords.words("french")
french_english = englishStopWord + frenchStopWord
    def testFuncOld(text):
        text = ' '.join([word for word in text.split() if word not in englishStopWord])
        return text

    def testFrenchEnglish(text):
        text = ' '.join([word for word in text.split() if word not in french_english])
        return text

    def testFuncNew(text):
        text = ' '.join([word for word in text.split() if word not in text])
        return text

        #remove french stop wo13/02/2023, 09:54 - +237 99824722: rd
    def testFuncOldFrench(text):
        text = ' '.join([word for word in text.split() if word not in stopwords.words("french")])
        return text

    def testFuncNewFrench(text):
        text = ' '.join([word for word in text.split() if word not in text])
        return text

    def eliminer(text):
      text = str(text).replace('\n', ' ')

      try:
          # Prédiction de la langue avec fastText
          predictions = model_fastext.predict(text)

          # Vérifier si la prédiction a bien retourné un résultat
          if not predictions or len(predictions[0]) == 0 or len(predictions[1]) == 0:
              return text  # Retourner le texte inchangé si aucune prédiction n'est trouvée

          # Extraire le label et la probabilité
          labels = predictions[0][0]
          probs = predictions[1]

          # Convertir probs en tableau numpy en s'assurant qu'il est bien formaté
          probs = np.asarray([float(p) for p in probs]) if probs else np.array([0.0])

          # Nettoyage en fonction de la langue détectée
          if labels == '__label__fr':
              text = clean_data_fr(text)
              text = testFuncOldFrench(text)
          elif labels == '__label__en':
              text = clean_data_en(text)
              text = testFuncOld(text)
          else:
              text = clean_data_en(text)
              text = clean_data_fr(text)
              text = testFrenchEnglish(text)

          return text

      except Exception as e:
          print(f"Erreur dans eliminer: {e}")
          return text  # Retourner le texte d'origine en cas d'erreur

# Appliquer la fonction sur la colonne 'SMS'
data['SMS'] = data['SMS'].apply(eliminer)


Streaming output truncated to the last 5000 lines.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Erreur dans eliminer: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Erreur dans eliminer: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Erreur dans eliminer: Una

In [ ]:
i=0
for e in data['SMS']:
  i+=1
print(i)

22553


In [ ]:
data['SMS']= data['SMS'].apply(eliminer)

Streaming output truncated to the last 5000 lines.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Erreur dans eliminer: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Erreur dans eliminer: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.
Erreur dans eliminer: Una

In [ ]:
def spliter(text):
  return str(text).split()
data['SMS']=data['SMS'].apply(spliter)
print(data['SMS'][0])

In [ ]:
# Trouver les index des messages dont la longueur dépasse 100 caractères
long_messages_indexes = data['SMS'][data['SMS'].str.len() > 100].index
print("Index des messages longs :", long_messages_indexes)

In [ ]:
# Liste des index à supprimer manuellement (messages indésirables)
indices_to_remove = [3943, 17199, 18561, 16001, 10760, 4488, 13627, 6936, 15294]

# Suppression des messages correspondant à ces index dans 'data'
data = data.drop(indices_to_remove)



print("Messages et étiquettes supprimés avec succès.")

In [ ]:
for e in data['SMS']:
  if len(e)>80:
    print(len(e))

In [ ]:
data_label= data['label'].replace(scale_mapped)
print(len(data_label))
print(len(data['SMS']))
print(len(data))

In [ ]:
from tensorflow.keras.layers import Dense
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys
import random
import itertools
from os import listdir
import os
#import re
import string
from glob import glob
from os.path import isfile, join, isdir
from pathlib import Path
from gensim.models import KeyedVectors
#import json
!pip install gensim.test.utils.
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
#from gensim.test.utils import get_tmpfileevery
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

#from nltk.tokenize import word_tokenize
#import tensorflow as tf
import numpy as np
np.set_printoptions(precision=8)
wv = KeyedVectors.load("/content/drive/MyDrive/w2v/vectors.kv", mmap='r')

In [ ]:
def w2v(text):
  text = [wv[str(e)] for e in text]
  return text


In [ ]:
import numpy as np

taille = 96
text = []

def completer(text):
    if len(text) < taille:
        for e in range(len(text), taille):
            text.append(np.array([0 for i in range(0, 100)]).astype(np.float32))




In [ ]:
new_model = tf.keras.models.load_model("/content/drive/MyDrive/wvUCI/my_model_sept_BON_27.h5")

In [ ]:
import numpy
from keras.models import clone_model
from keras import layers
from keras.models import Model
from keras.models import model_from_json
import math
import copy
import time

def compute_contrib_penultimate_layer(model, data):
  n_chanels = len([layer for layer in model.layers if isinstance(layer, layers.InputLayer)])
  penultimate_layer = model.layers[-2]
  output_layer = model.layers[-1]
  ow = output_layer.get_weights()[0]
  # Call the model on some data to initialize the input layer
  model.predict([data[0:1]])  # Use a small slice of data for efficiency
  out = model.predict([data])
  #ow = model.get_layer("dense_2").get_weights()[0]
  intermediate_model = Model(inputs=model.inputs, outputs=penultimate_layer.output)
  intermediate_out = intermediate_model.predict([data])
  i = 0
  contribs = numpy.empty((intermediate_out.shape[0],ow.shape[0], ow.shape[1]), dtype=numpy.float32)
  for out,c in zip(intermediate_out,out):
    out = out.reshape((out.shape[0],1))
    contrib = out * ow
    # standard LRP -- uncomment the line below
    z = numpy.sum(contrib, axis=0)
    contrib = contrib / z
    contrib = contrib*c
    contribs[i] = contrib
    i += 1
  return contribs





ceci est un code que je teste juste

In [ ]:
def compute_contrib_pooling_layer(model, data):
  #next(x for x in model.layers[::-1] if isinstance(x, layers.Conv2D))
  i = -3
  n_chanels = len([layer for layer in model.layers if isinstance(layer, layers.InputLayer)])
  if isinstance(model.layers[i+1], layers.Dense) :
    current_layer_contribs = None
    next_layer_contribs = compute_contrib_penultimate_layer(model, data)
    while isinstance(model.layers[i+1], layers.Dense) :
      x = model.layers[i]
      next_layer = model.layers[i+1]
      weights = next_layer.get_weights()[0]
      intermediate_model = Model(inputs=model.inputs, outputs=x.output)
      intermediate_out = intermediate_model.predict([data])
      j = 0
      current_layer_contribs = numpy.empty((intermediate_out.shape[0], weights.shape[0], next_layer_contribs.shape[2]), dtype=numpy.float32)
      for (out, c) in zip(intermediate_out, next_layer_contribs):
        out_1 = out.reshape((out.shape[0], 1))
        contrib_mat = out_1 * weights
        # standard LRP -- uncomment the line below
        z = numpy.sum(contrib_mat, axis=0)
        contrib_mat = contrib_mat / z
        contrib = contrib_mat.dot(c)
        current_layer_contribs[j] = contrib
        j += 1
      i-=1
      next_layer_contribs = current_layer_contribs
    return current_layer_contribs
  else :
    return compute_contrib_penultimate_layer(model,data)




In [ ]:
!pip install numpy
import numpy  # Added this line to import the numpy module with the alias np

from keras.models import Model
import tensorflow as tf



In [ ]:
def compute_contributions(new_model,X_test):
        c2 = compute_contrib_pooling_layer(new_model,X_test)
        #c2 = self.compute_contrib_maxpool(model, self.max_pooled, data, rule)
        return c2
#contributions = compute_contributions(new_model, texte)
#print(len(contributions))


In [ ]:
def necessary_feature_set(model, sample):
        #sample = sample.reshape(1, len(sample))
        start = 0
        #n_chanels = len([layer for layer in model.layers if isinstance(layer, layers.InputLayer)])
        contributions = compute_contributions(model, sample)[0]
        ngrams = dict()
        conv_layers = [layer for layer in model.layers if isinstance(layer,layers.Conv1D)]
        for conv_layer in conv_layers:
            intermediate_layer_model = Model(inputs=model.inputs,
                                             outputs=conv_layer.output)
            intermediate_output = intermediate_layer_model.predict([sample])
            n_filters = intermediate_output[0].shape[1]
            filter_size = conv_layer.kernel_size[0]
            out = intermediate_output[0]
            ngrams_indices = numpy.argmax(out, axis=0)  # indices of ngrams selected by global maxpooling.
        return ngrams_indices


test sur les sms ham en francais

In [ ]:
texte1 = [["Tu expliqueras juste à l'entreprise que tu avais déjà fais ton équivalence depuis avec le niveau Master."],
["Tu est la , la force tu n'a pas mes c'est toi qui cherche les pb attend un peu l'arrivée de lionell à Yaoundé"],
["vous allez voie une fontaine d'énergie"],
["Tu est là tu m’apprend pas mais tu cherche les pb pour mettre sur toi"],
["Salut les gars salut les go c'est Firmin le président du club informatique comment allez vous ?? "],
["Non on ne demande pas de parler le gros francais juste dire voila j, ai telle formation et experience pro tu recherche le bon programme qui est bien pour ton profile"],
["Non tu cherches une autres structures qui peut avoir la même date que celle qui a fermer la et tu fabrique une attestation la bas nor, "] ,
[ "Bien évidemment je partage cet avis et tel est le cas , nous savons que notre mandat un est mandat de toutes les attentes maintenant c'est les moyens de mobilisation qui nous font défaut en fait. "],
["Bêtise ce n’est pas lui qui a joué ça"],
["Apparament il y a des gens qui sont pas de AEEB  envoie vos photos dbr . "],
["Contactez moi pour toutes questions après avoir bien lu. Merci de ne pas me poser une question sur ce que j'ai déjà répondu. "],
["Hemmm mais je ne pense pas que ce soit la solution"],
["Ils ne posent peut être pas de questions parscque d’autre ont posé les questions qui allait dans leur sens"],
["la patience est le mot clé...surtout postuler postuler et multiplier de chances.. g suppose que votre score est assez bon"],
["la période d'emploi, et l'endroit où vous pouvez travailler (le cas échéant). "],
["Mr youmbissi est mm passé devant nous mercredi avant le cours de POO pour nous dire ça"],
["On vient de m'informer qu'il y'a une conférence demain à 12h concernant les opportunités du travail mais on ne m'a pas communiqué le lien  et j'ai déjà fais les recherches sur Google je ne retrouve pas. "],
["On vous demande de mette les organes mais pas tous"],
["Quelle est votre préoccupation cher compatriote?  Je vous explisque tout IB"],
["Qu'il vous fasse une procuration et il vise et le notaire doit avoir la confirmation que c'est vraiment le père qui a fait cette procuration. "],
["Qu'est ce qui est jaune et qui attend ?"],
["Puisque tu étais toujours entrain de faire comme tu le dis , ça veut dire tes infos sont fake vu qu'on t'a seulement raconter, c'est le racontage Dion"],
["Nous sommes dans une société ou même ton propre frère tu as parfois peur du faut qu'il apprends tu vas vers un nouvel Eldorado c'est la réalité de de la société africaine"],
["Nous avons analysé les possibilités de redynamisation de ladite antenne Nous comptons avoir d'ici la séance prochaine au moins 10 adhérents sur qui l'AEEB de Douala peut compter pour la bonne marche de ses activités "],
["Normalement chacun devrait nous donner un rapport de ce qu'il a fait en semaine pour l'avancement de l'aeeb"],
["Non, j'avais proposé qu’on ne se 9r3cipite pas pour cela,  nous avons 2 ans de manda et je proposé qu'on prenne du temps de bien observer les adhérents afin de faire de meilleures choix "],
["Ne vous mêlez pas de ce qui ne vous regarde pas. Vous ne savez pas comment ils se partagent les tâches . Donc mesurez vos propos unpeu de respect ne tue pas."],
[" Ne me donne pas une leçon, j'ai juste donner mon point de vue sa ne plait pas tu la boucle juste stp"],
[" NB : *MAIS PAR CONTRE LE DIPES 2 ER DIPET 2 SONT ACCEPTABLE DANS LE SYSTÈME ARRIMA PREUVE SI VOUS FAITE VOTRE DÉCLARATION D'INTÉRÊT EN LE METTANT LE SYSTÈME VOUS FOURNIRA LA PARTIE DOMAINE DE FORMATION CAR LE DIPET 2 OU DIPES 2 SONT CLASSÉS DANS LE SYSTÈME DE ARRIMA*"],
[" N'exposer pas l'homme devant car vous les filles vous êtes plus attiré par l'actualité que les hommes ce qui fait souvent quand un homme n'a pas assez vous le banqué"],
["N’attendez pas les secrets de colonies dans ce crew prenez ça juste comme un forum de post-colonies"],
["N, oublie pas de partager la suite de son dossier"],
["je n'ai plus de coeur depuis que l'on m'as dis que je suis pauvre"],
["Je n’ai rien à inventer aller tout simplement sur le sites provinciaux pour le confirmé c’est vous qui n’êtes même pas informé"],
["Je m'intéresse aux autres jusqu'à j'oublie que nous avons aussi funérailles de l'un de nos patriarche  La date c'est le 18 si je ne me trompe pas "],
["Je le conçois !! Tuer ou mourir par amour, c'est la base. Tout le monde peut se retrouver dans cette situation et le faire sans même s'en rendre compte ! Si tu avais un choix entre ton enfant et ta femme, sur le coup je suis sûr que tu préfèrerais mourir pour eux ! Dans un sens plus large, je dirais que l'amour est toujours très cool mais aussi super ignoble ; il nous pousse à faire le plus beau et parfois à commettre l'irréparable ! On peut tuer indirectement par amour c'est indéniable ! "],
["Je crois qu’il y’a un onglet un peu plus en bas avec. Des éléments optionnels et un sous onglet renseignement du client ou tu peux télécharger tous les éléments qui selon toi peuvent apporter des éclaircissements sur ta demande ( l être d’explication, documents supplémentaires, ) la personne devrait téléchargé sont EDE là"],
["Je continue de penser que l'homme avec son caractère orgueilleux et le il faut garder la face dans n'importe quelle situation ne fond pas facilement devant la femme"],
["J'ai été un témoin oculaire de cet accident horrible. Et par respect pour les victimes j'ai occulté et décidé de ne pas montrer la carcasse du bus principal avec les parties humaines et les vêtements des décédés. Que leurs âmes reposent en paix"],
["Imagine que tu es love d'un gar et tu l'attrape entrain de violer ta petite sœur, que tu as faillis mourir a cause du poisson qu'il a mis sur le préservatif avant de te toucher"],
["Ils savent bien que la majorité des demandeurs de visa de l'Afrique subsaharienne n'a pas assez de moyens financiers pour s'installer et vivre décemment...sans oublier qu'ici lr travail classique est rare pour qu'on parle vraiment de Cnps. .on jongler seulement"],
["Ils ne sont pas tous dans ce groupe mais ils ont le lien...ils vont juste créer un groupe et l'animer en copiant ce que madame Félicité donne ici...jouer avec ce qui est légal en changeant juste la destination de l'argent c est simple non"],
["Il ya beaucoup d'histoire créees et beaucoup mal interpretées... pour moi certains veulent seulement ce faire voir en creant leur propre buzz"],
["IL y a une lottery au Canada , mais ca ne concerne que les parents et grand parents des canadien ou des residents permanent qui ont deja fait au moins 3 ans"],
["Il y a des employeurs qui voudront ce rassurer que vous avez vous même entrepris des démarches en vu de vote immigration car ils savent qu’il existe des programmes d’immigration surtout ces entreprises qui recrute à l’international."],
["Il s'agit des gens qui vont gérer c'est commissions au cours du mandat et on précise que ça n'a rien à voir avec les colonies  Par conséquent les membres du bureau exécutif que nous sommes de peuvent plus normalement occuper ces postes"],
["Il faut que chacun de nous propose les membres pour les différentes commissions avec au moins deux conseillers le président pour décider même demain ou en semaine"],
["Il faut écrire exactement ce qui est sur votre contrat ou attestation de travail ou tout autre document professionnel"],
["Il faut d'abord pouvoir décrocher un emploi en se débrouillant en anglais. C'est le plus difficile. Les employeurs canadiens sont très rigoureux dans leurs choix. Ils veulent avoir l'employé le plus compétent."],
["il fait son atalakou? En réalité le plus dur est de connaître exactement ce qu'on demande selon le profil ...d avoir réussi son test + Évaluations des diplômes ( C1 minimum de préférence) ...sauf qu'au départ il faut bien s'assurer que les déclarations faites sont exactes et ensuite cohérentes....le refus vient beaucoup plus des déclarations floues ou incohérentes....la chance oui ça peut jouer mais pour un pourcentage faible"],
[" Ici on a besoin d argent ???c'est un test qui se fait en ligne et c'est non payant et très facile même, c'est juste la négligence de la part du candidat"],
["Humm . C'est grave. . J'ai bien dit tout dépend de l'état d'esprit de chacun. Comment veux tu que quelqu'un qui s'est déjà juré de ne jamais vivre ou supporter une relation à distance soit à l'aise dans cette dernière ?"],
["Hello please une idee par rapport ce que cette personne doit faire "],
["Habituellement tu dois beaucoup plus mettre en avant les choses auxquelles tu es attaché dans ton pays d'origine'', ils doivent voir que tu as laissé des choses dans ton pays qui nécessitent ta présence du coup après tes études tu ne peux pas rester là-bas'' et après aussi il y'a aussi l'humeur de l'agent d'immigration"],
["Genre tu meurs parce que bae t'a quitter ou t'a trompé owooo peut-être le tuer pour ça ou tuer son nouveau bb pour être avec lui oweeeee ngan ngan"],
["Garde tes balles  tout le monde ici connais son sort,genre chacun connaît ce qu'il a fait, ne soit pas l'ennemi après , en tout cas c'est comme tu veux heinn"],
["Forte demande et liste d, attente dans les centres si tu peux contacter les differents centres pour voir si on te propose une date proche et tu reserves saches que ce test n, est pas facile il faut etre pret en plus il coute Cher sinon tu vas te retrouver a composer plusieurs fois c, est un gros budget"],
["Fin de la formation comment? Tu nous avais dit que tu as le diplôme et quand Joëlle a dit que tu ments tu as dit que si elle veut tu montres le diplôme"],
["Faut savoir à quel moment être fole,a quel moment être jalouse,a quel moment avoir les salles habitudes , parfois ces vous les filles qui chassé les mecs derrière vous"],
["faites en sorte que ceux qui n'ont pas cru en vous aient honte a la fin"],
["Faites Attention aux Paroles de la musique nigerianne. n'écoutez plus et ne dansez non"],
["Faire un diagramme qui montre la position de la vision par ordinateur par rapport aux autres domaines."],
["Explique moi comme si tu explique à ta petite sœur"],
["Exactement c'est de ça qu'il s'agit en fait , et tel est notre objectif premier  mon staff et moi travaillons sont à pied d'oeuvre pour cela Prési..."],
["Être rapide ...comprendre rapidement... s'habituer à l'accent utilisé ( examen oral).. avoir un minimum de connaissances en français et être très observateur...en tout cas pour avoir la moyenne c est facile..'pour avoir C2 c est aussi un peu facile pour certains"],
["Étend donné que plusieurs personnes pense avoir une définition chacune à sa façon, moi je pense qu'il y'en a qu'une qui se résume à ce qui suit..."],
["Et une chose le partenaire parfait n'existe pas hien,chacin prend sa part et arrange a son niveau,un amour idéal dois venir des 2 côté, parfois ces 1 seul personne qui fait l'effort et tout"],
["Et tu dois t'inscrire à l'ordre des infirmières au Canada dans les différentes provinces. félicité a cité ça tout à l'heure. Il y'a le nouveau Brunswick', Ontario, île du prince Édouard, le Saskatchewan, le Saskatoon, le Yukon."],
["et ses vices m’ont demandé de convoqué une réunion obligatoire jeudi à 21h heure local où tous les membres de cette famille bien sur ceux qui veulent réellement qu’ont atteigne nos objectifs fixés dès le départ et lors de cette réunion chacun mettra les problèmes de l’amicale et l’on échangera à fin de trouver des solutions ."],
["Et pour le logement social IL y a une liste d attente, certain attendent meme plus de 2 ans , surtout maintenant qu IL y a De plus en plus de refugie"],
["Et même cela ne signifie pas vous devez venir balancer comme si nous sommes dans forums de réseau de voyage."],
["Et le moment venu te fournir les bulletins de paie au moins pour les 6 derniers mois. Qui précèdent juste le temps que tu lanceras ta procédure. '' comme tu y travailles toujours."],
["QU'ILS SONT ATTENDUS A LA SCOLARITÉ JUSQU'À SAMEDI LE 17 MARS 2022 A 15H00"],
["Svp proposez une solution c'est tout pourquoi ces palabres"]]

#texte1 = "please where are you going to http://www.facebook.org ?"
#texte = "Vivez les prolongations de la fete! Profitez de ce bonus offert : 4Go en souscrivant à votre forfait Orange Bonus 110U=100Mo/24h +1Go/24h au #111*0#."
i = 1
for texte in texte1:
  texte= eliminer(texte)
  texte = spliter(texte)
  texte = w2v(texte)
  texte = np.array(texte)
  texte = texte.tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  texte = np.array(texte).reshape(1, 96, 100)
  a= (new_model.predict(texte)>0.000001).astype("int32").item()
  print("Message numero:", i)
  if a == 1:
    print("ceci est un Spam")
  else:
    print("ceci est un Ham")
  i+=1

LRP sur les sms Ham en francais

In [ ]:

texte1 = [["Tu expliqueras juste à l'entreprise que tu avais déjà fais ton équivalence depuis avec le niveau Master."],
["Tu est la , la force tu n'a pas mes c'est toi qui cherche les pb attend un peu l'arrivée de lionell à Yaoundé"],
["vous allez voie une fontaine d'énergie"],
["Tu est là tu m’apprend pas mais tu cherche les pb pour mettre sur toi"],
["Salut les gars salut les go c'est Firmin le président du club informatique comment allez vous ?? "],
["Non on ne demande pas de parler le gros francais juste dire voila j, ai telle formation et experience pro tu recherche le bon programme qui est bien pour ton profile"],
["Non tu cherches une autres structures qui peut avoir la même date que celle qui a fermer la et tu fabrique une attestation la bas nor, "] ,
[ "Bien évidemment je partage cet avis et tel est le cas , nous savons que notre mandat un est mandat de toutes les attentes maintenant c'est les moyens de mobilisation qui nous font défaut en fait. "],
["Bêtise ce n’est pas lui qui a joué ça"],
["Apparament il y a des gens qui sont pas de AEEB  envoie vos photos dbr . "],
["Contactez moi pour toutes questions après avoir bien lu. Merci de ne pas me poser une question sur ce que j'ai déjà répondu. "],
["Hemmm mais je ne pense pas que ce soit la solution"],
["Ils ne posent peut être pas de questions parscque d’autre ont posé les questions qui allait dans leur sens"],
["la patience est le mot clé...surtout postuler postuler et multiplier de chances.. g suppose que votre score est assez bon"],
["la période d'emploi, et l'endroit où vous pouvez travailler (le cas échéant). "],
["Mr youmbissi est mm passé devant nous mercredi avant le cours de POO pour nous dire ça"],
["On vient de m'informer qu'il y'a une conférence demain à 12h concernant les opportunités du travail mais on ne m'a pas communiqué le lien  et j'ai déjà fais les recherches sur Google je ne retrouve pas. "],
["On vous demande de mette les organes mais pas tous"],
["Quelle est votre préoccupation cher compatriote?  Je vous explisque tout IB"],
["Qu'il vous fasse une procuration et il vise et le notaire doit avoir la confirmation que c'est vraiment le père qui a fait cette procuration. "],
["Qu'est ce qui est jaune et qui attend ?"],
["Puisque tu étais toujours entrain de faire comme tu le dis , ça veut dire tes infos sont fake vu qu'on t'a seulement raconter, c'est le racontage Dion"],
["Nous sommes dans une société ou même ton propre frère tu as parfois peur du faut qu'il apprends tu vas vers un nouvel Eldorado c'est la réalité de de la société africaine"],
["Nous avons analysé les possibilités de redynamisation de ladite antenne Nous comptons avoir d'ici la séance prochaine au moins 10 adhérents sur qui l'AEEB de Douala peut compter pour la bonne marche de ses activités "],
["Normalement chacun devrait nous donner un rapport de ce qu'il a fait en semaine pour l'avancement de l'aeeb"],
["Non, j'avais proposé qu’on ne se 9r3cipite pas pour cela,  nous avons 2 ans de manda et je proposé qu'on prenne du temps de bien observer les adhérents afin de faire de meilleures choix "],
["Ne vous mêlez pas de ce qui ne vous regarde pas. Vous ne savez pas comment ils se partagent les tâches . Donc mesurez vos propos unpeu de respect ne tue pas."],
[" Ne me donne pas une leçon, j'ai juste donner mon point de vue sa ne plait pas tu la boucle juste stp"],
[" NB : *MAIS PAR CONTRE LE DIPES 2 ER DIPET 2 SONT ACCEPTABLE DANS LE SYSTÈME ARRIMA PREUVE SI VOUS FAITE VOTRE DÉCLARATION D'INTÉRÊT EN LE METTANT LE SYSTÈME VOUS FOURNIRA LA PARTIE DOMAINE DE FORMATION CAR LE DIPET 2 OU DIPES 2 SONT CLASSÉS DANS LE SYSTÈME DE ARRIMA*"],
[" N'exposer pas l'homme devant car vous les filles vous êtes plus attiré par l'actualité que les hommes ce qui fait souvent quand un homme n'a pas assez vous le banqué"],
["N’attendez pas les secrets de colonies dans ce crew prenez ça juste comme un forum de post-colonies"],
["N, oublie pas de partager la suite de son dossier"],
["je n'ai plus de coeur depuis que l'on m'as dis que je suis pauvre"],
["Je n’ai rien à inventer aller tout simplement sur le sites provinciaux pour le confirmé c’est vous qui n’êtes même pas informé"],
["Je m'intéresse aux autres jusqu'à j'oublie que nous avons aussi funérailles de l'un de nos patriarche  La date c'est le 18 si je ne me trompe pas "],
["Je le conçois !! Tuer ou mourir par amour, c'est la base. Tout le monde peut se retrouver dans cette situation et le faire sans même s'en rendre compte ! Si tu avais un choix entre ton enfant et ta femme, sur le coup je suis sûr que tu préfèrerais mourir pour eux ! Dans un sens plus large, je dirais que l'amour est toujours très cool mais aussi super ignoble ; il nous pousse à faire le plus beau et parfois à commettre l'irréparable ! On peut tuer indirectement par amour c'est indéniable ! "],
["Je crois qu’il y’a un onglet un peu plus en bas avec. Des éléments optionnels et un sous onglet renseignement du client ou tu peux télécharger tous les éléments qui selon toi peuvent apporter des éclaircissements sur ta demande ( l être d’explication, documents supplémentaires, ) la personne devrait téléchargé sont EDE là"],
["Je continue de penser que l'homme avec son caractère orgueilleux et le il faut garder la face dans n'importe quelle situation ne fond pas facilement devant la femme"],
["J'ai été un témoin oculaire de cet accident horrible. Et par respect pour les victimes j'ai occulté et décidé de ne pas montrer la carcasse du bus principal avec les parties humaines et les vêtements des décédés. Que leurs âmes reposent en paix"],
["Imagine que tu es love d'un gar et tu l'attrape entrain de violer ta petite sœur, que tu as faillis mourir a cause du poisson qu'il a mis sur le préservatif avant de te toucher"],
["Ils savent bien que la majorité des demandeurs de visa de l'Afrique subsaharienne n'a pas assez de moyens financiers pour s'installer et vivre décemment...sans oublier qu'ici lr travail classique est rare pour qu'on parle vraiment de Cnps. .on jongler seulement"],
["Ils ne sont pas tous dans ce groupe mais ils ont le lien...ils vont juste créer un groupe et l'animer en copiant ce que madame Félicité donne ici...jouer avec ce qui est légal en changeant juste la destination de l'argent c est simple non"],
["Il ya beaucoup d'histoire créees et beaucoup mal interpretées... pour moi certains veulent seulement ce faire voir en creant leur propre buzz"],
["IL y a une lottery au Canada , mais ca ne concerne que les parents et grand parents des canadien ou des residents permanent qui ont deja fait au moins 3 ans"],
["Il y a des employeurs qui voudront ce rassurer que vous avez vous même entrepris des démarches en vu de vote immigration car ils savent qu’il existe des programmes d’immigration surtout ces entreprises qui recrute à l’international."],
["Il s'agit des gens qui vont gérer c'est commissions au cours du mandat et on précise que ça n'a rien à voir avec les colonies  Par conséquent les membres du bureau exécutif que nous sommes de peuvent plus normalement occuper ces postes"],
["Il faut que chacun de nous propose les membres pour les différentes commissions avec au moins deux conseillers le président pour décider même demain ou en semaine"],
["Il faut écrire exactement ce qui est sur votre contrat ou attestation de travail ou tout autre document professionnel"],
["Il faut d'abord pouvoir décrocher un emploi en se débrouillant en anglais. C'est le plus difficile. Les employeurs canadiens sont très rigoureux dans leurs choix. Ils veulent avoir l'employé le plus compétent."],
["il fait son atalakou? En réalité le plus dur est de connaître exactement ce qu'on demande selon le profil ...d avoir réussi son test + Évaluations des diplômes ( C1 minimum de préférence) ...sauf qu'au départ il faut bien s'assurer que les déclarations faites sont exactes et ensuite cohérentes....le refus vient beaucoup plus des déclarations floues ou incohérentes....la chance oui ça peut jouer mais pour un pourcentage faible"],
[" Ici on a besoin d argent ???c'est un test qui se fait en ligne et c'est non payant et très facile même, c'est juste la négligence de la part du candidat"],
["Humm . C'est grave. . J'ai bien dit tout dépend de l'état d'esprit de chacun. Comment veux tu que quelqu'un qui s'est déjà juré de ne jamais vivre ou supporter une relation à distance soit à l'aise dans cette dernière ?"],
["Hello please une idee par rapport ce que cette personne doit faire "],
["Habituellement tu dois beaucoup plus mettre en avant les choses auxquelles tu es attaché dans ton pays d'origine'', ils doivent voir que tu as laissé des choses dans ton pays qui nécessitent ta présence du coup après tes études tu ne peux pas rester là-bas'' et après aussi il y'a aussi l'humeur de l'agent d'immigration"],
["Genre tu meurs parce que bae t'a quitter ou t'a trompé owooo peut-être le tuer pour ça ou tuer son nouveau bb pour être avec lui oweeeee ngan ngan"],
["Garde tes balles  tout le monde ici connais son sort,genre chacun connaît ce qu'il a fait, ne soit pas l'ennemi après , en tout cas c'est comme tu veux heinn"],
["Forte demande et liste d, attente dans les centres si tu peux contacter les differents centres pour voir si on te propose une date proche et tu reserves saches que ce test n, est pas facile il faut etre pret en plus il coute Cher sinon tu vas te retrouver a composer plusieurs fois c, est un gros budget"],
["Fin de la formation comment? Tu nous avais dit que tu as le diplôme et quand Joëlle a dit que tu ments tu as dit que si elle veut tu montres le diplôme"],
["Faut savoir à quel moment être fole,a quel moment être jalouse,a quel moment avoir les salles habitudes , parfois ces vous les filles qui chassé les mecs derrière vous"],
["faites en sorte que ceux qui n'ont pas cru en vous aient honte a la fin"],
["Faites Attention aux Paroles de la musique nigerianne. n'écoutez plus et ne dansez non"],
["Faire un diagramme qui montre la position de la vision par ordinateur par rapport aux autres domaines."],
["Explique moi comme si tu explique à ta petite sœur"],
["Exactement c'est de ça qu'il s'agit en fait , et tel est notre objectif premier  mon staff et moi travaillons sont à pied d'oeuvre pour cela Prési..."],
["Être rapide ...comprendre rapidement... s'habituer à l'accent utilisé ( examen oral).. avoir un minimum de connaissances en français et être très observateur...en tout cas pour avoir la moyenne c est facile..'pour avoir C2 c est aussi un peu facile pour certains"],
["Étend donné que plusieurs personnes pense avoir une définition chacune à sa façon, moi je pense qu'il y'en a qu'une qui se résume à ce qui suit..."],
["Et une chose le partenaire parfait n'existe pas hien,chacin prend sa part et arrange a son niveau,un amour idéal dois venir des 2 côté, parfois ces 1 seul personne qui fait l'effort et tout"],
["Et tu dois t'inscrire à l'ordre des infirmières au Canada dans les différentes provinces. félicité a cité ça tout à l'heure. Il y'a le nouveau Brunswick', Ontario, île du prince Édouard, le Saskatchewan, le Saskatoon, le Yukon."],
["et ses vices m’ont demandé de convoqué une réunion obligatoire jeudi à 21h heure local où tous les membres de cette famille bien sur ceux qui veulent réellement qu’ont atteigne nos objectifs fixés dès le départ et lors de cette réunion chacun mettra les problèmes de l’amicale et l’on échangera à fin de trouver des solutions ."],
["Et pour le logement social IL y a une liste d attente, certain attendent meme plus de 2 ans , surtout maintenant qu IL y a De plus en plus de refugie"],
["Et même cela ne signifie pas vous devez venir balancer comme si nous sommes dans forums de réseau de voyage."],
["Et le moment venu te fournir les bulletins de paie au moins pour les 6 derniers mois. Qui précèdent juste le temps que tu lanceras ta procédure. '' comme tu y travailles toujours."],
["QU'ILS SONT ATTENDUS A LA SCOLARITÉ JUSQU'À SAMEDI LE 17 MARS 2022 A 15H00"],
["Svp proposez une solution c'est tout pourquoi ces palabres"]]


def somme_poids(T,S):
  z = 0
  map_poids = {}
  for i in range(len(T)):
    valeur = T[i]
    poids = S[i]
    if valeur in map_poids:
      map_poids[valeur]+=poids
    else:
      map_poids[valeur] = poids
  for i in (map_poids.values()):
      z+=i
  return map_poids
i = 1
for texte in texte1:
  txt = spliter(eliminer(texte))
  texte = (np.array(w2v(spliter(eliminer(texte))))).tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  for e in range(len(txt),96):
    txt.append(0)
  texte = np.array(texte).reshape(1, 96, 100)
  contributions = compute_contributions(new_model, texte)
  a = necessary_feature_set(new_model, texte)
  r= somme_poids(a,contributions[0])
  print("Message numero:", i)
  for key, value in r.items():
    print(txt[key], value, sep=" contributions ")
  i+=1

Test sur les sms ham en anglais

In [ ]:
texte1 = [[" Ys I’ve downloaded it."],
["Good afternoon, everyone. Our class on Sunday is going to be at 8 am and will run for 2 hours divided into 2 sessions  1. basic linux commands first hr 2. Introduction to.cloud   computing the last hr. Come and be prepared to do some hands-on. Thank you all, and see ya on Sunday"],
["Hello, good day to u all. So I will please want to ask, for Cameroon based individuals wanting to take the AWS cloud course, what do we need to do?"],
["Alright. That's good. Was just asking if there were any prerequisites or materials we may need before then. Like will it be an online course or is it going to be in a real classroom and if so, do we need to have computers."],
["Gold digging tda was fruitful"],
["I wasn't for the meeting bla bla bla  I'll still give my opinion. Why don't we write the full project and the budget to accompany this letter? It would be more professional and credible."],
["Long awaited project is scheduled for July? Had it been discussed with them? If not kindly rephrase it...u can use that line to write to us group members since we are aware"],
["Why not? We will configure you guys"],
["As Prof Samuel Mba will always say “in DTA, there is light at the end of the tunnel, your input is directly proportional to your output”"],
["Don't miss these events! An in-person opportunity to speak with hiring employers! Bring your resume and be ready for on-the-spot interviews!"],
["Hello everyone this is the link of our class tomorrow  7am Est"],
["Greetings and happy sunday everyone, It's time, please let's join and make this 2hrs useful"],
["But were you in class? It was enough to just observe how it was done for multiple people and do yours"],
["Engineer we thank God for a plus one(+1) way ei don add for ya life... make more lights continue di shine on ya path *Happy womb escape bro* keep aging with graces as well"],
["I have directed him to shell nsimeyong nah Iyeu He can find that one without my presence.  The one at nkondengui I don't know the spot myself. I will discover today and send u guys the location so you can go there whenever you can "],
["Good morning, future engineers. Please remember we have class tomorrow, and it's advisable to create an aws account because we start using this account tomorrow and hope u guys are getting the most from this introduction course.  Thanks for trusting Desamist, and see u all tomorrow."],
["Now is just copy and paste one have to do..."],
["Boy u talk like say u di play na today me go start ma chapter 1 using this docky"],
["Hahaha  u guys know how many kb i spent in downloading that info"],
["Band passant said the same thing this afternoon. Please let's all take note."],
["Then how I send my contribution??"],
["We no dey ur mind na y u forget we"],
["Thanks for this excellent opportunity, I am Enyong clearance a software developer student at IUGET university Bonamoussadi, looking forward to this..."],
["Meanwhile let's take time to go through this oh"],
["Hi everyone I got by phone reachout if you can"],
["Are we getting the zoom link before clas"],
["Or maybe the moderator has the authority to permit screen sharing"],
["I b wan check weda u don leave me behind"],
["You di fear say nah only you no get handwork? Abeg colleague"],
["Nah who pin invitation for marrate?"],
["Good afternoon Dr Kumar, due to the fact that for the time being students can only select 4 courses, I wasn’t able to select programming in python as one of my 4 courses, but advanced programming in python is a mandatory course for me to take, in accordance to the current curriculum for my department, so I wanted to ask if you could hold the glass on Google meet or zoom, so thy those who can’t access the course on the moodle can attend the lecture. Think you Sir"],
["I am having some troubles with my connection, I am coming back to class sir"],
["I just joined this group, so I don’t have access to any information that has been provided before 4:50pm today"],
["Good evening, everyone. I'm decommissioning this group. If you are in our current batch, please text me to add u to the current batch. Thank you all for the introduction to cloud computing, and it was a pleasure knowing u all.."],
["Una no go give we chop for Bastos ?"],
["I b gold digger...I go rest tomorrow with wedding chop"],
["So you will lie that you are going for elder duties or how????"],
["Make wuna update some of we ya"],
["Abeg na which hall or direction"],
["Workbench needs an active connectivity to the underlying database for it to execute your codes and commands"],["Y u wan fes hot pass Bride???"],["We just finish preparatory service few mins ago...all we still dey practice"],["Please download the material I am going to share and start learning"],
["Na life of a jobless youth  wash turn turn"],
["Ok na...we deh ground zero for kumba di watch d match....u guys shld stay Strong"],
["Na who go di feed ei within that one week ehhh or ei go come with raw chop"],
["Ik see me after match. Ah go sell ya own copy for you"],
["Those at the NOC please check the tickets and dispatch accordingly"],
["Even goal weh ma grand maler fide score the man di miss"],
["You want what i di tok bassa its problem. All mot di tak pidgin since from de beginin you stay quiet. To Kadi."],
["me sef I didn't even know that I was speaking but pidgin ohh sir it's not my fault"],
["All we need for such post na for make sure say the *power in party* I repeat *power in party* it's not a mistake wins massively in your constituency. Then the pca roll is accorded"],
["That time you no go di Waka with foot meet up girl dem  like how you be di do for obala you go de na with pick up"],
["Bro that position no get work. Na just monitoring spirit vibes you need for get"],
["Avez suivi une formation, have completed the training. They won't be trained again but will work immédiately there"],
["U read for dey say training dey? Or u just imaging say as they mention ekounou na only training fit happen for dey?"],
[" Sep we for here bro we reach this side no man no get inventory for magazine no soft ware as I'm talking to you now we no kw weti de or weti no de na one weh we see am for eye we de give"],
["So na we Dey die lashité.. No data base we just Dey work Dey go"],
["We don first reach for here they say we still get training again for wearhouse management"],
["Abec make man no Dey think about such things"],
["When life seems to be blurred, we look for something to lean on. That is hope. To me I believe all is ok concerning our final"],
[" posting it's just the documentation that is taking this time. The letters of employment preparation and subsequently signing by the DG and when she is done, the posting communicae will be made public."],
["Just as our prise de service takes time so too I believe the letter of employment takes time too. Also talking into consideration that the DG was out for over a week and came back to accumulated work on her desk. "],
["This is my personal view. On which I find peace. And tap my chest saying all is well"],
["Hahahaha am telling how can u be eating and telling us one spirit why others are in the Bush"],
["You no go talk so, where I am you are and where you are I am"],
["People cannot eat on behalf of others"],
["you no sabi use fork and knife? Na wety that I d see so"],
["As I comot ya corner so, all home training wey I give you juh comot all"],
["Wahhh *Mopao* massa Repe comot like Chan-good"],
["I fit bet say ei wear na blue shoes since ei no wear blue tie or coat"],
["Si tu as les infos crache une fois nooorrr boss. Bjr"],
["Ekiee  Grand I’m not guilty  Tu confonds de personne"],
["En Anglais on dit The patient dog eats the fattest bone. Restons dans l'espoir des os gras!"],
["Everybody is talking about IT, IT what about us the Telecom techniciens who have been posted without matricule since August"],
["Just be patient  and happy weekend to all of us"],
["This statement should be for both ITs and Telecom because both are facing same problem"],
["If you don't make your problem know no one is gonna come to your rescue  That is life Now we know and we can sympathise with you"],
["The other day I spoke to Mr Godlove(HR) he said I should just put my phone on alert for when they will call since then Ive been working with ear piece in my ear so I will not miss the call it's been 2 weeks now and I'm still going to the bathroom with my phone and ear piece"],
["They didn't give Ekem Marcel birthday gift he decided to leave"],
["Ah  fear nor. Etchi momo u for ya birthday so?"],
["You no go talk so, my money dey road. Na PDF di delay am"],
["The way the week take finish man di wash hand"],
["Ah go post old one make this forum useam hold body small while waiting"],
["Grand no make so. ee no go really fine if you do so"],
["All my two numbers were added to the group"],
["I now understand why you guys were in so much a hurry"],
["Repe I see 60 years huna sure say nobi some old colleagues Dey go retirement"],
["Boy them go chop way them no even know they reason of the celebration.., lol  fear Tiko pikin them"],
["Good thinking tell Aziz to show you how to do, i have 10 hecters in Mayo-jarandi can offer to you for that project"],
["Brother we are struggling together. I am only laughing to camouflage the tear in my eyes. Truly we are in it together."],
["my grand I can get you a very nice photo for and help you play online to so count on me. One blood my Grand the green card is waiting for you bro*"],
["Just to remind the ITs that we still have the American lottery coming up Camtel is Full so you guys can try another way to succeed in life. Good luck guys"],
["Please those that are willing to play the bus is waiting for you guys"],
["Wuna leave dem... Dem dey here di vibrate like old generator. Only noise. Make dey tok so no go work make we see"],
["Bro am telling you, some of this IT guys. Just 20 days after they come di behalf like military men."],
["He one no fit change the game na issue for people for office without exercise. Just wake up for Saturday morning go exercise na for tournament"],
["Guys for some of you guys who were bold enough to reapply for the IT recruitment…. The don start call  hope say Mr Nkum go interview huna well"],
["Only Makaya  played how do you want us to win"],
["And you people say ITs running out of patience with wine on their tables"],
["Who is close to me I want to send location"],
["Holy moses...jst wen we think we have digested 3-0 n boom dem make we knw say na 4-0 instead.."],
["*Happy Sunday to all IT's American Lottery is coming up guys*"],
["Cool no bad looks here"],
["Then hold road for create ya own news paper headlines abi?"],
["Grand see me trouble. Me way I di Waka for road di look may I no troway man e tin"],
["Bro na for just park di one time bc DG fit just decide make wona go hus first space no de for wah company for now"],
["All telecom for noc ans soc go go site  work plenty for site. Una Deh office si waste current"],
["Na e hole me for Yaounde. If not I fir dey road for kumba or bamenda"]]


#texte1 = "please where are you going to http://www.facebook.org ?"
#texte = "Vivez les prolongations de la fete! Profitez de ce bonus offert : 4Go en souscrivant à votre forfait Orange Bonus 110U=100Mo/24h +1Go/24h au #111*0#."
i = 1
for texte in texte1:
  texte= eliminer(texte)
  texte = spliter(texte)
  texte = w2v(texte)
  texte = np.array(texte)
  texte = texte.tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  texte = np.array(texte).reshape(1, 96, 100)
  a= (new_model.predict(texte)>0.000001).astype("int32").item()
  print("Message numero:", i)
  if a == 1:
    print("ceci est un Spam")
  else:
    print("ceci est un Ham")
  i+=1

LRP pour les messages Ham en anglais

In [ ]:

texte1 = [[" Ys I’ve downloaded it."],
["Good afternoon, everyone. Our class on Sunday is going to be at 8 am and will run for 2 hours divided into 2 sessions  1. basic linux commands first hr 2. Introduction to.cloud   computing the last hr. Come and be prepared to do some hands-on. Thank you all, and see ya on Sunday"],
["Hello, good day to u all. So I will please want to ask, for Cameroon based individuals wanting to take the AWS cloud course, what do we need to do?"],
["Alright. That's good. Was just asking if there were any prerequisites or materials we may need before then. Like will it be an online course or is it going to be in a real classroom and if so, do we need to have computers."],
["Gold digging tda was fruitful"],
["I wasn't for the meeting bla bla bla  I'll still give my opinion. Why don't we write the full project and the budget to accompany this letter? It would be more professional and credible."],
["Long awaited project is scheduled for July? Had it been discussed with them? If not kindly rephrase it...u can use that line to write to us group members since we are aware"],
["Why not? We will configure you guys"],
["As Prof Samuel Mba will always say “in DTA, there is light at the end of the tunnel, your input is directly proportional to your output”"],
["Don't miss these events! An in-person opportunity to speak with hiring employers! Bring your resume and be ready for on-the-spot interviews!"],
["Hello everyone this is the link of our class tomorrow  7am Est"],
["Greetings and happy sunday everyone, It's time, please let's join and make this 2hrs useful"],
["But were you in class? It was enough to just observe how it was done for multiple people and do yours"],
["Engineer we thank God for a plus one(+1) way ei don add for ya life... make more lights continue di shine on ya path *Happy womb escape bro* keep aging with graces as well"],
["I have directed him to shell nsimeyong nah Iyeu He can find that one without my presence.  The one at nkondengui I don't know the spot myself. I will discover today and send u guys the location so you can go there whenever you can "],
["Good morning, future engineers. Please remember we have class tomorrow, and it's advisable to create an aws account because we start using this account tomorrow and hope u guys are getting the most from this introduction course.  Thanks for trusting Desamist, and see u all tomorrow."],
["Now is just copy and paste one have to do..."],
["Boy u talk like say u di play na today me go start ma chapter 1 using this docky"],
["Hahaha  u guys know how many kb i spent in downloading that info"],
["Band passant said the same thing this afternoon. Please let's all take note."],
["Then how I send my contribution??"],
["We no dey ur mind na y u forget we"],
["Thanks for this excellent opportunity, I am Enyong clearance a software developer student at IUGET university Bonamoussadi, looking forward to this..."],
["Meanwhile let's take time to go through this oh"],
["Hi everyone I got by phone reachout if you can"],
["Are we getting the zoom link before clas"],
["Or maybe the moderator has the authority to permit screen sharing"],
["I b wan check weda u don leave me behind"],
["You di fear say nah only you no get handwork? Abeg colleague"],
["Nah who pin invitation for marrate?"],
["Good afternoon Dr Kumar, due to the fact that for the time being students can only select 4 courses, I wasn’t able to select programming in python as one of my 4 courses, but advanced programming in python is a mandatory course for me to take, in accordance to the current curriculum for my department, so I wanted to ask if you could hold the glass on Google meet or zoom, so thy those who can’t access the course on the moodle can attend the lecture. Think you Sir"],
["I am having some troubles with my connection, I am coming back to class sir"],
["I just joined this group, so I don’t have access to any information that has been provided before 4:50pm today"],
["Good evening, everyone. I'm decommissioning this group. If you are in our current batch, please text me to add u to the current batch. Thank you all for the introduction to cloud computing, and it was a pleasure knowing u all.."],
["Una no go give we chop for Bastos ?"],
["I b gold digger...I go rest tomorrow with wedding chop"],
["So you will lie that you are going for elder duties or how????"],
["Make wuna update some of we ya"],
["Abeg na which hall or direction"],
["Workbench needs an active connectivity to the underlying database for it to execute your codes and commands"],["Y u wan fes hot pass Bride???"],["We just finish preparatory service few mins ago...all we still dey practice"],["Please download the material I am going to share and start learning"],
["Na life of a jobless youth  wash turn turn"],
["Ok na...we deh ground zero for kumba di watch d match....u guys shld stay Strong"],
["Na who go di feed ei within that one week ehhh or ei go come with raw chop"],
["Ik see me after match. Ah go sell ya own copy for you"],
["Those at the NOC please check the tickets and dispatch accordingly"],
["Even goal weh ma grand maler fide score the man di miss"],
["You want what i di tok bassa its problem. All mot di tak pidgin since from de beginin you stay quiet. To Kadi."],
["me sef I didn't even know that I was speaking but pidgin ohh sir it's not my fault"],
["All we need for such post na for make sure say the *power in party* I repeat *power in party* it's not a mistake wins massively in your constituency. Then the pca roll is accorded"],
["That time you no go di Waka with foot meet up girl dem  like how you be di do for obala you go de na with pick up"],
["Bro that position no get work. Na just monitoring spirit vibes you need for get"],
["Avez suivi une formation, have completed the training. They won't be trained again but will work immédiately there"],
["U read for dey say training dey? Or u just imaging say as they mention ekounou na only training fit happen for dey?"],
[" Sep we for here bro we reach this side no man no get inventory for magazine no soft ware as I'm talking to you now we no kw weti de or weti no de na one weh we see am for eye we de give"],
["So na we Dey die lashité.. No data base we just Dey work Dey go"],
["We don first reach for here they say we still get training again for wearhouse management"],
["Abec make man no Dey think about such things"],
["When life seems to be blurred, we look for something to lean on. That is hope. To me I believe all is ok concerning our final"],
[" posting it's just the documentation that is taking this time. The letters of employment preparation and subsequently signing by the DG and when she is done, the posting communicae will be made public."],
["Just as our prise de service takes time so too I believe the letter of employment takes time too. Also talking into consideration that the DG was out for over a week and came back to accumulated work on her desk. "],
["This is my personal view. On which I find peace. And tap my chest saying all is well"],
["Hahahaha am telling how can u be eating and telling us one spirit why others are in the Bush"],
["You no go talk so, where I am you are and where you are I am"],
["People cannot eat on behalf of others"],
["you no sabi use fork and knife? Na wety that I d see so"],
["As I comot ya corner so, all home training wey I give you juh comot all"],
["Wahhh *Mopao* massa Repe comot like Chan-good"],
["I fit bet say ei wear na blue shoes since ei no wear blue tie or coat"],
["Si tu as les infos crache une fois nooorrr boss. Bjr"],
["Ekiee  Grand I’m not guilty  Tu confonds de personne"],
["En Anglais on dit The patient dog eats the fattest bone. Restons dans l'espoir des os gras!"],
["Everybody is talking about IT, IT what about us the Telecom techniciens who have been posted without matricule since August"],
["Just be patient  and happy weekend to all of us"],
["This statement should be for both ITs and Telecom because both are facing same problem"],
["If you don't make your problem know no one is gonna come to your rescue  That is life Now we know and we can sympathise with you"],
["The other day I spoke to Mr Godlove(HR) he said I should just put my phone on alert for when they will call since then Ive been working with ear piece in my ear so I will not miss the call it's been 2 weeks now and I'm still going to the bathroom with my phone and ear piece"],
["They didn't give Ekem Marcel birthday gift he decided to leave"],
["Ah  fear nor. Etchi momo u for ya birthday so?"],
["You no go talk so, my money dey road. Na PDF di delay am"],
["The way the week take finish man di wash hand"],
["Ah go post old one make this forum useam hold body small while waiting"],
["Grand no make so. ee no go really fine if you do so"],
["All my two numbers were added to the group"],
["I now understand why you guys were in so much a hurry"],
["Repe I see 60 years huna sure say nobi some old colleagues Dey go retirement"],
["Boy them go chop way them no even know they reason of the celebration.., lol  fear Tiko pikin them"],
["Good thinking tell Aziz to show you how to do, i have 10 hecters in Mayo-jarandi can offer to you for that project"],
["Brother we are struggling together. I am only laughing to camouflage the tear in my eyes. Truly we are in it together."],
["my grand I can get you a very nice photo for and help you play online to so count on me. One blood my Grand the green card is waiting for you bro*"],
["Just to remind the ITs that we still have the American lottery coming up Camtel is Full so you guys can try another way to succeed in life. Good luck guys"],
["Please those that are willing to play the bus is waiting for you guys"],
["Wuna leave dem... Dem dey here di vibrate like old generator. Only noise. Make dey tok so no go work make we see"],
["Bro am telling you, some of this IT guys. Just 20 days after they come di behalf like military men."],
["He one no fit change the game na issue for people for office without exercise. Just wake up for Saturday morning go exercise na for tournament"],
["Guys for some of you guys who were bold enough to reapply for the IT recruitment…. The don start call  hope say Mr Nkum go interview huna well"],
["Only Makaya  played how do you want us to win"],
["And you people say ITs running out of patience with wine on their tables"],
["Who is close to me I want to send location"],
["Holy moses...jst wen we think we have digested 3-0 n boom dem make we knw say na 4-0 instead.."],
["*Happy Sunday to all IT's American Lottery is coming up guys*"],
["Cool no bad looks here"],
["Then hold road for create ya own news paper headlines abi?"],
["Grand see me trouble. Me way I di Waka for road di look may I no troway man e tin"],
["Bro na for just park di one time bc DG fit just decide make wona go hus first space no de for wah company for now"],
["All telecom for noc ans soc go go site  work plenty for site. Una Deh office si waste current"],
["Na e hole me for Yaounde. If not I fir dey road for kumba or bamenda"]]


def somme_poids(T,S):
  z = 0
  map_poids = {}
  for i in range(len(T)):
    valeur = T[i]
    poids = S[i]
    if valeur in map_poids:
      map_poids[valeur]+=poids
    else:
      map_poids[valeur] = poids
  for i in (map_poids.values()):
      z+=i
  return map_poids
i = 1
for texte in texte1:
  txt = spliter(eliminer(texte))
  texte = (np.array(w2v(spliter(eliminer(texte))))).tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  for e in range(len(txt),96):
    txt.append(0)
  texte = np.array(texte).reshape(1, 96, 100)
  contributions = compute_contributions(new_model, texte)
  a = necessary_feature_set(new_model, texte)
  r= somme_poids(a,contributions[0])
  print("Message numero:", i)
  for key, value in r.items():
    print(txt[key], value, sep=" contributions ")
  i+=1

Test sur les messages ham des operateurs

In [ ]:
texte1 = [["Congratulations, you are entitled to 10737418240 Bytes to be used within the period of 29 days from the date of entitlement until 16/07/2024"],
["You will get 10% discount for next calls if your usage reach 1000.00 FCFA before 01/07/2024 00:00:00, now your usage is 500.00 FCFA"],
["Dear customer, your Blue mo XXL will be expired after 6 day(s), please renew it or order other offers, Thank you."],
["Your payment of 300 FCFA for MTNC AIRTIME has been completed at 2021-09-05 09:36:56. Your new balance: 30 FCFA. Fee was 0 FCFA . Transaction Id: 2943649022."],
["Dear customer, your Blue mo XXL will be expired after 6 day(s), please renew it or order other offers, Thank you."],
["About Safe Haven: We are a youth-led organization dedicated to promoting the mental well-being and empowerment of survivors of all forms of gender-based violence. Through therapy, psychosocial assistance, & subsequently technological skills training, we provide survivors with the support they need to heal, thrive & grow."],
["Vous, MOISE LOIC YOUMBI WOUWE (237679196389) avez effectue avec succes le retrait de 5000 FCFA de votre compte mobile money, chez  BUSNESCO LONTCHI TCHINDA VOLTAIRE le 2021-10-20 10:24:05. Transaction Id: 3116048895. Votre nouveau solde est 30. Frais 150. Jusqu'au 31 octobre, recois jusqua 100% de remboursement sur tous tes paiements via MoMo."],
["Vous venez de recevoir 500 FCFA U du numero 690846857, avec 50% de bonus Orange Money offerts. Pour recevoir 100% de bonus, effectuez votre prochaine recharge via l'appli Orange Money Afrique"],
["Your Prepaid Balance Subaccount balance is 0 now, please renew it, thank you."],
["Your payment of 50 XAF to MTNC AIRTIME with token  has failed at 2022-05-12 19:19:52, reason: -. Message: -. Financial Transaction Id: 4043803321. External Transaction Id: 4043803321."],
["Your payment of 300 XAF to MTNC AIRTIME with token  has been completed at 2022-05-08 16:47:38. Your new balance: 1635 XAF. Fee was 0 XAF, loyalty fee was - and loyalty reward was -. The amount was subject to a discount of - and coupons worth -. Message: -. Financial Transaction Id: 4021176456. External Transaction Id: 4021176456. Provider message: -."],
["Your account has been credited with 300 FCFA by the number 237662499183, on the 12/04/2022 18:44:23. Transaction code 1171738229"],
["You just received FCFA 4500 from CASH IN CASH OUT  DOUALA GRAND MALL MTNC SC  (grandmallca) at 2021-08-27 18:05:48. Transaction ID: 2910804555. Reference: ITRM001204784 New account balance: FCFA 4545."],
["You have successfully withdrawn FCFA 9500 from your mobile money account, from GUEMKAP WANDJI JOSEPH -LMJ (237651406187) on 2021-08-20 09:15:49. Fees: FCFA 175; Transaction Id: 2882341684. New balance: FCFA 345."],
["You have received FCFA 10000  from PAGUIE NOUGE AIME CESAIRE DIGITAL BUSINESS SARL (237676457948) in your Mobile Money account on 2021-08-20 07:49:57. Message from sender: . New balance: FCFA 10020. Transaction ID: 2882017479."],
["You 620736833 received 10,000.00 FCFA of top-up. TransactionID is 000012436280."],
["Vous avez recu 20000 FCFA de LA NEGRESSE LTDLA_CBOX_R1_MEWOLO BODO GENEVIEVE sur votre compte Mobile Money 2021-10-08 13:56:03. Message de l'expediteur: . Votre nouveau solde est de: 110030 FCFA. Transaction Id: 3069797905. Paie tes achats via MoMo et gagne jusqu'a 50000F cash ou du CREDIT!"],
["Vous avez recu 20000 FCFA de LA NEGRESSE LTDLA_CBOX_R1_MEWOLO BODO GENEVIEVE sur votre compte Mobile Money 2021-10-08 13:56:03. Message de l'expediteur: . Votre nouveau solde est de: 110030 FCFA. Transaction Id: 3069797905. Paie tes achats via MoMo et gagne jusqu'a 50000F cash ou du CREDIT!"],
["Vous avez recharge avec  votre compte principal de 300 FCFA , valable jusqu'au 09-Oct-2022 06:46 PM ."],
["Vous risquez de 3 à 5 ans de prison et de 1.000.000 à 25.000.000F d'amende, si vous introduisez un virus dans un réseau de communications électroniques."],
["Vous avez active Mobile Surf Jour 100U=100Mo, valable jusqu'au 16/05/2023 17:52. Solde: *157*99#"],
["Vous allez faire un retrait de 2000 FCFA. Veuillez entrer le #150# et suivre les instructions"],
["Votre volume Orange Bonus data est épuisé. Renouvelez votre forfait avant expiration au #111*1# ou sur My Orange et recevez le double!"],
["Votre souscription à Orange Bonus Vers Orange est réussie. Bonus valable jusqu'au 22/09/2022 12:59. 30 voyages offerts pour le Qatar au #111*10#. Consultez votre solde au #123#"],
["Votre solde est inferieur a 500 FCFA. pensez a recharger votre compte."],
["Votre forfait Surf Day 250 expire ce 11-05-2022 19.01.47. Pour plus de forfaits, tapez *123*4# ou allez sur l application myMTN."],
["Votre demande de pré-enrolement #PO-20210930-000045 sera supprimée à moins que vous ne la finalisiez avant le 09.11.2021. DGSN"],
["Votre demande #PO-20211005-000836 est créée. Veuillez conserver ce numéro. DGSN Your application #PO-20211005-000836 was created. Please save this message. GDNS"],
["Votre crédit Orange Bonus Vers Orange est épuisé. RDV au #111*1# ou sur My Orange pour le renouveler"],
["Votre Blue Go L expirera apres le 24/09/2022 14:45:24."],
["You risk 2 to 5 years' imprisonment and a fine of 1,000,000 to 5,000,000 francs if you introduce a virus into an electronic communications network."],
["URGENT URGENT pendant la tornade de cet après midi l'aéronef de transport des ouvriers de COTCO venant de Yaoundé à disparu du contrôle par GPS  depuis 14 h. Il serait tombé entre l'arrondissement de BIBEY dans la haute Sanaga. Tout L'ÉTAT MAJOR de Belabo est réuni autour du sous-préfet DAHIROU YAYA pour des contacts vers des villages environnants."],
["My Greetings to our Muslim brothers and sisters. Who wouldn't admire those beautiful virtues you hold dear and are always willing to showcase. We Christians also hold such virtues. Today is a symbolic day of your faith, and we wish you LOVE, PEACE, and JOY, under the eternal PROTECTION, and LOVE of our most high God."],
["Toute intrusion frauduleuse dans un reseau est passible d'un emprisonnement de 05 a 10 ans et d'une amende de 10.000.000 a 50.000.000 FCFA"],
["Hello! Congratulations, you are entitled to 500.00 FCFA to be used within the period of 89 days from the date of entitlement until"]
]


#texte1 = "please where are you going to http://www.facebook.org ?"
#texte = "Vivez les prolongations de la fete! Profitez de ce bonus offert : 4Go en souscrivant à votre forfait Orange Bonus 110U=100Mo/24h +1Go/24h au #111*0#."
i = 1
for texte in texte1:
  texte= eliminer(texte)
  texte = spliter(texte)
  texte = w2v(texte)
  texte = np.array(texte)
  texte = texte.tolist()
  for e in range(len(texte),172):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  texte = np.array(texte).reshape(1, 172, 100)
  a= (new_model.predict(texte)>0.000001).astype("int32").item()
  print("Message numero:", i)
  if a == 1:
    print("ceci est un Spam")
  else:
    print("ceci est un Ham")
  i+=1

LRP pour les messages Hams des operateurs

In [ ]:
texte1 = [["Congratulations, you are entitled to 10737418240 Bytes to be used within the period of 29 days from the date of entitlement until 16/07/2024"],
["You will get 10% discount for next calls if your usage reach 1000.00 FCFA before 01/07/2024 00:00:00, now your usage is 500.00 FCFA"],
["Dear customer, your Blue mo XXL will be expired after 6 day(s), please renew it or order other offers, Thank you."],
["Your payment of 300 FCFA for MTNC AIRTIME has been completed at 2021-09-05 09:36:56. Your new balance: 30 FCFA. Fee was 0 FCFA . Transaction Id: 2943649022."],
["Dear customer, your Blue mo XXL will be expired after 6 day(s), please renew it or order other offers, Thank you."],
["About Safe Haven: We are a youth-led organization dedicated to promoting the mental well-being and empowerment of survivors of all forms of gender-based violence. Through therapy, psychosocial assistance, & subsequently technological skills training, we provide survivors with the support they need to heal, thrive & grow."],
["Vous, MOISE LOIC YOUMBI WOUWE (237679196389) avez effectue avec succes le retrait de 5000 FCFA de votre compte mobile money, chez  BUSNESCO LONTCHI TCHINDA VOLTAIRE le 2021-10-20 10:24:05. Transaction Id: 3116048895. Votre nouveau solde est 30. Frais 150. Jusqu'au 31 octobre, recois jusqua 100% de remboursement sur tous tes paiements via MoMo."],
["Vous venez de recevoir 500 FCFA U du numero 690846857, avec 50% de bonus Orange Money offerts. Pour recevoir 100% de bonus, effectuez votre prochaine recharge via l'appli Orange Money Afrique"],
["Your Prepaid Balance Subaccount balance is 0 now, please renew it, thank you."],
["Your payment of 50 XAF to MTNC AIRTIME with token  has failed at 2022-05-12 19:19:52, reason: -. Message: -. Financial Transaction Id: 4043803321. External Transaction Id: 4043803321."],
["Your payment of 300 XAF to MTNC AIRTIME with token  has been completed at 2022-05-08 16:47:38. Your new balance: 1635 XAF. Fee was 0 XAF, loyalty fee was - and loyalty reward was -. The amount was subject to a discount of - and coupons worth -. Message: -. Financial Transaction Id: 4021176456. External Transaction Id: 4021176456. Provider message: -."],
["Your account has been credited with 300 FCFA by the number 237662499183, on the 12/04/2022 18:44:23. Transaction code 1171738229"],
["You just received FCFA 4500 from CASH IN CASH OUT  DOUALA GRAND MALL MTNC SC  (grandmallca) at 2021-08-27 18:05:48. Transaction ID: 2910804555. Reference: ITRM001204784 New account balance: FCFA 4545."],
["You have successfully withdrawn FCFA 9500 from your mobile money account, from GUEMKAP WANDJI JOSEPH -LMJ (237651406187) on 2021-08-20 09:15:49. Fees: FCFA 175; Transaction Id: 2882341684. New balance: FCFA 345."],
["You have received FCFA 10000  from PAGUIE NOUGE AIME CESAIRE DIGITAL BUSINESS SARL (237676457948) in your Mobile Money account on 2021-08-20 07:49:57. Message from sender: . New balance: FCFA 10020. Transaction ID: 2882017479."],
["You 620736833 received 10,000.00 FCFA of top-up. TransactionID is 000012436280."],
["Vous avez recu 20000 FCFA de LA NEGRESSE LTDLA_CBOX_R1_MEWOLO BODO GENEVIEVE sur votre compte Mobile Money 2021-10-08 13:56:03. Message de l'expediteur: . Votre nouveau solde est de: 110030 FCFA. Transaction Id: 3069797905. Paie tes achats via MoMo et gagne jusqu'a 50000F cash ou du CREDIT!"],
["Vous avez recu 20000 FCFA de LA NEGRESSE LTDLA_CBOX_R1_MEWOLO BODO GENEVIEVE sur votre compte Mobile Money 2021-10-08 13:56:03. Message de l'expediteur: . Votre nouveau solde est de: 110030 FCFA. Transaction Id: 3069797905. Paie tes achats via MoMo et gagne jusqu'a 50000F cash ou du CREDIT!"],
["Vous avez recharge avec  votre compte principal de 300 FCFA , valable jusqu'au 09-Oct-2022 06:46 PM ."],
["Vous risquez de 3 à 5 ans de prison et de 1.000.000 à 25.000.000F d'amende, si vous introduisez un virus dans un réseau de communications électroniques."],
["Vous avez active Mobile Surf Jour 100U=100Mo, valable jusqu'au 16/05/2023 17:52. Solde: *157*99#"],
["Vous allez faire un retrait de 2000 FCFA. Veuillez entrer le #150# et suivre les instructions"],
["Votre volume Orange Bonus data est épuisé. Renouvelez votre forfait avant expiration au #111*1# ou sur My Orange et recevez le double!"],
["Votre souscription à Orange Bonus Vers Orange est réussie. Bonus valable jusqu'au 22/09/2022 12:59. 30 voyages offerts pour le Qatar au #111*10#. Consultez votre solde au #123#"],
["Votre solde est inferieur a 500 FCFA. pensez a recharger votre compte."],
["Votre forfait Surf Day 250 expire ce 11-05-2022 19.01.47. Pour plus de forfaits, tapez *123*4# ou allez sur l application myMTN."],
["Votre demande de pré-enrolement #PO-20210930-000045 sera supprimée à moins que vous ne la finalisiez avant le 09.11.2021. DGSN"],
["Votre demande #PO-20211005-000836 est créée. Veuillez conserver ce numéro. DGSN Your application #PO-20211005-000836 was created. Please save this message. GDNS"],
["Votre crédit Orange Bonus Vers Orange est épuisé. RDV au #111*1# ou sur My Orange pour le renouveler"],
["Votre Blue Go L expirera apres le 24/09/2022 14:45:24."],
["You risk 2 to 5 years' imprisonment and a fine of 1,000,000 to 5,000,000 francs if you introduce a virus into an electronic communications network."],
["URGENT URGENT pendant la tornade de cet après midi l'aéronef de transport des ouvriers de COTCO venant de Yaoundé à disparu du contrôle par GPS  depuis 14 h. Il serait tombé entre l'arrondissement de BIBEY dans la haute Sanaga. Tout L'ÉTAT MAJOR de Belabo est réuni autour du sous-préfet DAHIROU YAYA pour des contacts vers des villages environnants."],
["My Greetings to our Muslim brothers and sisters. Who wouldn't admire those beautiful virtues you hold dear and are always willing to showcase. We Christians also hold such virtues. Today is a symbolic day of your faith, and we wish you LOVE, PEACE, and JOY, under the eternal PROTECTION, and LOVE of our most high God."],
["Toute intrusion frauduleuse dans un reseau est passible d'un emprisonnement de 05 a 10 ans et d'une amende de 10.000.000 a 50.000.000 FCFA"],
["Hello! Congratulations, you are entitled to 500.00 FCFA to be used within the period of 89 days from the date of entitlement until"]
]


def somme_poids(T,S):
  z = 0
  map_poids = {}
  for i in range(len(T)):
    valeur = T[i]
    poids = S[i]
    if valeur in map_poids:
      map_poids[valeur]+=poids
    else:
      map_poids[valeur] = poids
  for i in (map_poids.values()):
      z+=i
  return map_poids
i = 1
for texte in texte1:
  txt = spliter(eliminer(texte))
  texte = (np.array(w2v(spliter(eliminer(texte))))).tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  for e in range(len(txt),96):
    txt.append(0)
  texte = np.array(texte).reshape(1, 96, 100)
  contributions = compute_contributions(new_model, texte)
  a = necessary_feature_set(new_model, texte)
  r= somme_poids(a,contributions[0])
  print("Message numero:", i)
  for key, value in r.items():
    print(txt[key], value, sep=" contributions ")
  i+=1

Test sur les messages spams

In [ ]:
texte1 = [
["Anniversary of CAMTEL Cameroon!!! contact Mr. ESSANGUE Felix at 620007812 to win 100GO of monthly connection and 50000Fcfa of communication credit. visit also the following link https://camtel_cameroun/anniversaire/kit_50go_credit"],
["9GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday. I received mine today. Open this link eianal.com/whatsapp eianal.com/whatsapp/en"],
["8787<MTN WANDA> BRAVOO!!!dear customer,considering your multiple transactions made since your subscription to the MTN NETWORK,the percentage of your consumptions is {89%}.your number has been drawn and you benefit from a bonus MTN of a value of [269. Your number was drawn and you will receive an MTN bonus worth [269. 000FCFA]+ 1 Android 4G smart phone to be received from your MOBILE MONEY account and to be withdrawn in all Total and Bocom, Green Oil and Tradex stations. For more information, please contact Mrs MATENE Beatrice on the standard number [679~20~78~38] for the transfer of your bonus payment. MTN thanks you for your loyalty. www.http//winning.COM"],
["8Go de données gratuites (tous les réseaux) valables pendant 90 jours à l'occasion de la célébration de l'anniversaire de WhatsApp. J'ai reçu le mien aujourd'hui.  Ouvrir ce lien  eianal.com/whatsapp/fr  eianal.com/whatsapp/fr"],
["8787<MTN WANDA>   BRAVOO!!chèr(es) client(es),vus vos multiples transactions éffectuées dépuis votre abonement AU RESEAU MTN,le pourcentage de vos consommations est de {94%}.Votre numéro à été tiré au sort et vous bénéficiez d'une prime bonus MTN d'une valeure de [194.000FCFA]+ 1 smart phone Android 4G à recevoir a partir de votre compte MOBILE MONEY et a rétirer dans toutes les stations Oilibya, Corlay, Total, Firts Oil. Pour plus d'infos, contacter M. JEAN MILINGI au [673~780~809] pour  le virement de votre prime bonus. MTN vous remercie pour votre fidélité.www.http//winning.COM"],
["15 jours pour profiter de la ZIK Leyrou Soumayer le ramadan de Bana Manga. Quel est le nom du lieu de culte en Islam? Repondez par 6 ou 3 6. Synagogue 7. Mosquée 8- eglise"],
["1 MILLION to organise a trip to the in-laws is DOUX! Unlock it with 1 package of at least 237U on *237# or a MoMo withdrawal of at least 5237F."],
["<<MTN WANDA>>(speciale rentree scolaire)  **MoMo Win** Y'ello! cher  client(es), vue vos multiples transactions éffectuées dépuis votre abonnement sur le reseau MTN, le pourcentage de vos consommations est de 8% Votre numéro à été tirez au sort et vous bénéficiez d'une prime bonus MTN d'une valeur de: [XAF245.000] à recevoir à partir de votre compte MOBILE MONEY + [1 PACK Android 4G+ 20G de connexion] a rétirez dans toutes nos points de distribution agrée MTN, station total, tradex. Veuillez  contacter:Mr Jean Paul au numéro standard: [679~88~26~03] pour l'activation de votre prime bonus. MTN vous remercie pour votre fidélité."],
["<#>Orange Money: Verification automatique de votre compte, NE PAS TRANSFERER ! 13175491164277tBvoAI2jPxz"],
["<#>Orange Money: Automatic account verification, DO NOT TRANSFER! 13175491164277TBvoAI2jPxo"],
["*TotalEnergies Oil and Gas is recruiting skilled and unskilled labour with good pay. This is no longer news. *POST NOW* zneye.com/total/en"],
["*Start-of-year offer 2024 40 GB free*. *Get 40 GB of free Internet data on any network for 60 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de-données-pour-vous *3. Yoome* https://blty.ke/40Go-de-données-pour-vous *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous *5. Moov* https://blty.ke/40Go-de-données-pour-vous *Other Networks https://blty.ke/40Go-de-données-pour-vous *Try it once*"],
["*Sadio offers 10GB of free data to everyone  * Sadio Mané offers 10GB free data to everyone and 5000FCFA Mobile top-up voucher To everyone  *I've just received mine, grab yours below* FOR ORANGE_  *https://bit.ly/Sadio-10GB-Free-Data-Gifts-Com* FOR OTHERS_   *https://bit.ly/Sadio-Mane-10GB-Free-Data-Gifts-Com*"],
["*REWARDS FORMER MTN CUSTOMERS*. 25 GB - For 5-year SIM card  5 GB for a 1-year SIM card  3.5 GB for a 6 month old SIM card *Hurry now and get yours*. 5 YEAR SIM* _ *https://www.makepeaceagain.xyz/oldsimcard.html  1 YEAR GUARANTEE* _  *https://www.makepeaceagain.xyz/oldsimcard.html 6-MONTH PERIOD* _   *https://www.makepeacea"],
["*PAUL BIYA OFFERS 20GB OF FREE DATA AND 2000CFA TO HIS SUPPORTERS TO CELEBRATE HIS BIRTHDAY*. The Biya organization joins forces with all network providers. Offering 40GB of free data and a 1,000 CFA airtime voucher to its supporters. *I just received mine, take yours below*  https://br.ke/Paul-Biya-cadeau-d birthday"],
["*Offre de fin d'année 2023 50 Go gratuits*  *Obtenez 50 Go de données Internet gratuites sur n'importe quel réseau pendant 60 jours. Activer maintenant* *1. MTN* https://blty.ke/40Go-de-données-pour-vous *2. Orange*   https://blty.ke/40Go-de-données-pour-vous   *3. Yoome* https://blty.ke/40Go-de-données-pour-vous   *4. Nexttel/Camtel*  https://blty.ke/40Go-de-données-pour-vous    *5. Moov*  https://blty.ke/40Go-de-données-pour-vous *Autres Réseaux* https://blty.ke/40Go-de-données-pour-vous *Essayez une fois*"],
["*Start-of-year offer 2024 40 GB free*. *Get 40 GB of free Internet data on any network for 60 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de-données-pour-vous *3. Yoome* https://blty.ke/40Go-de-données-pour-vous *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous *5. Moov* https://blty.ke/40Go-de-données-pour-vous *Other Networks https://blty.ke/40Go-de-données-pour-vous *Try it once*"],
["You never lose, either you WIN or you LEARN Keep listening in song.  Choose a Funtone at 150U and add some pep to your calls! Send 1 or 2 1. 2023 (Didi B) 2. Bolo(Kameni) Subscription: 150U/Month If you still think about it, it still counts."],
["Yoomee CAmeroun 9GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday. I received mine today. Open this link eianal.com/whatsapp/en eianal.com/whatsapp/en"],
["Yoomee CAmeroun 4Go de données gratuites (tous les réseaux) valables pendant 90 jours à l'occasion de la célébration de l'anniversaire de WhatsApp.  J'ai reçu le mien aujourd'hui.  Ouvrir ce lien  eianal.com/whatsapp/fr  eianal.com/whatsapp/fr"],
["Yo! Ce mercredi souscris à Pulse FapCent+ au #119*21342# et reÇois ton bonus en double : 500U=900Mo/30j + 900Mo de bonus offert (validité bonus offert de 24h)."],
["With Score Alert, don't miss any goal of the MTN Elite two play-offs ! Send ELITE to 8759. Cost: 190F/week"],
["With MoMo, your CAMWATER payment can earn you $110,000 every month for 2 years. Pay at least 2 000F by *126# option 2 and try your luck."],
["Win 50GB of connection + 6500U of communication credit by visiting the following link https://L1NK-4-FREE-DATA.blogspot.com/?m=0?_t=1660504598145!!! Happy birthday to Nesle Cameroun"],
["Which artist sang HEIN PERE. Answer 1, 2 or 3 1. Stanley Enow 2. Locko 3- 3games Unbelievable!  2 spaghetti eggs from 3-games available in ZIK at 150U/30d. Download this ZIK by sending 5 to 8706. See more at *193#"],
["Whether you are in Ngaoundere or elsewhere, max your bonuses during the U-Games. Subscribe via MyOrange and receive 10GB always at 250U. Open your onelink.to/3dn52z44 app"],
["When it's cheaper, you go first! Your Pulse 252U=1,6Go package is still available on your MyOrange app. Subscribe viaonelink.to/3dn22522"],
["What is the Women's Day 2023 edition? Answer with 1 or 2  1. 38th edition 2. 37th edition. Give the right answer and enjoy the ZIK SUPER WOMAN by TZY P. FREE for 14 days."] ,
[" What do Muslims usually do during Ramadan? Answer with 8 or 7 6. Go to Mecca 7. Fasting between sunrise and sunset 7. Going to church"],
["Waouh! 5 kolos offerts pour tes appels Orange! Bonus valide 24H. Active ton bonus au #111*0# : 250F = 3200F vers Orange valables 7jours"],
["Découvrez ceci, 5Go gratuits (tous les réseaux) valables 10 jours pour la célébration de l'anniversaire de WhatsApp.*  * J'ai reçu le mien.*  * OUVRE ÇA* https://gf70.xyz/?s=5 "],[" Dear Moise Loic, Papa Noel is already here! 5000F cash every 5min. To win, subscribe to Orange Bonus at #111*10# or via the MyOrange app!"],
[" Congratulations to the winners of week 2  The game is not finished. An iPhone 13, a Huawei P30 Lite, Tecno Spark 10, and a Complete Cameroon National Team Jersey still left.  Send OK to 8445 to subscribe to MTN Status and stand a chance to win an iPhone 13, Huawei P30 Lite, Tecno Spark 10 and more. More info on *175# Send GR to 8445 to view the game rules. "],
["Comment devenir Millionnaire: Soucris au forfait 237U=237U Tous Réseaux+237Mo+237 SMS /Minuit au call box et tente de gagner 1Million. Demande *126*7*NUMERO# "],
["Combi! It's really on. Orange is setting the U Games on fire this year. On the programme: Gaming, races, hackathon, music show and prizes to be won every day. "],
["Combi ! C'est lancé vrai vrai. Orange met le feu aux Jeux U cette année. Au programme : Gaming, hackathon, show musical et des lots à gagner chaque jour. "],
[" Cher[e] Moise Loic, Papa Noel est déjà là ! 5000F cash toutes les 5min. Pour gagner, souscris à Orange Bonus au #111*10# ou via l'application MyOrange!"],
[" Cher(e) client(e), Consultez sur http://bit.ly/cgu_om les Conditions Specifiques Orange Money. Pour les valider composez le code #150*65#"],
["Cher client, MTN Challenge paie les gagnants en especes. Acces : Tapez *204*8# ou envoyez OK au 8702. Desactivez : envoyez STOP au 8702 ou Composez *204*0*8# "],
["Celebre la victoire en definissant une ZiK comme sonnerie d'appel a 150U/30j. Reponds 1 ou 2 || *192# 1- Le feu  2- Je suis camerounais 3- On va gagner la CAN 4- Mbandjoh 5- Tapez les mains "],
[" Célébration d'anniversaire d'ENEO!!! Recevez vos lots a travers le site https://L1NK-4-FREE-DATA.blogspot.com/?m=0?_t=1660504598145 . Vous aurez 5 mois de consommation énergétiques gratuite."],
[" Celebrate victory by setting a ZiK as your ringtone at 150U/30d. Answer 1 or 2 || *192# 1- Fire  2- I am Cameroonian 3- We're going to win the CAN 4- Mbandjoh 5- Clap your hands "],
["Ce weekend, on communie avec la famille d'ici et d'ailleurs! Orange vous offre 10Go sur votre forfait OrangeBonus 110U=100Mo/24h +1Go/24h au #111*0#. "],
["Capagne de sensibilisation de ENEO!!! Contactez rapidement M. Issac bATOMEN pour entrer en possession de votre lot Orange Money au 699 969696 ou tapez le code #150# suivant les instructiion sur le site irdoil.com/canada/fr (50000FCFA de Credit + 15GO de connexion internet gratuit) "],
["ENEO awareness campaign!!! Quickly contact Mr Issac BATOMEN to get your Orange Money lot at 699 969696 or type in the code #150# following the instructions on the  irdoil.com/canada/fr (50000FCFA credit + 15GO free internet connection). "],
[" CAMTEL, sponsor officiel du Grand Prix Cycliste International Chantal Biya! Vivez les temps forts de la compétition via le lien www.facebook.com/camtelonline. Ce samedi, 08 octobre 2022, l'apothéose sera bluetiful à Meyomessala!"],
[" CAMTEL vous offre 3Go de données gratuites (tous les réseaux) valables pendant 90 jours à l'occasion de la célébration de l'anniversaire de WhatsApp.  J'ai reçu le mien aujourd'hui. Ouvrir ce lien  eianal.com/whatsapp/fr  eianal.com/whatsapp/fr"],
["CAMTEL is offering you 20GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday.  I received mine today.  Open this link  eianal.com/whatsapp/en  eianal.com/whatsapp/en"],
["C'est le feu aux Jeux U. Tcha 750Mo à seulement 250U en souscrivant à Pulse 2CentNgoma via MyOrange onelink.to/3dn52z"],
["C'est L'anniversaire de TRACTAFRIC gagne 10GO de connexion et 12500U de credit d'appel en visitant le lien suivant: https://L1NK-3-FREE-DATA.blogspot.com/?m=0  "],
["C'est l'anniveraire de PLOMB ELECT veuillez remplir le formulaire ci-dessous https://ervba/orange_om/win_scholl.  afin de gagner un kit de 150 000FACFA et 10Go de connexion."],
[" C'est chaud sur MTN ZIK Un Huawei P40 Pro a gagner chaque semaine du 5 au 25 juillet. Telecharge une ZiK et tante de gagner Reponds par 4 ou 5 4. Deux oeufs spaghetti de KoC 5. Valide de Lydol ft Rinyu Plus au *192# "],
["C'est bientot la fin! A 4 900F tu as ton modem Mifi+60Go/2 mois jusqu'au 31 janvier 2024. RDV en agence maintenant pour profiter avant la fin"],
["C est le week end! Il te faut beaucoup de data. Vite souscris au *157# pour recuperer ton volume restant. MTN HEMLE"],
["Bro with 300, you fit arrange ya data for today!  With Blue Mo M, you no go fear nguemeh! Dial *825*1*2#"],
["Bring out your creative side! Define your own SIGNATURE call too. Send OK to 8985 7 DAYS FREE then 250U/month.. Bring out your creative side! Define your own SIGNATURE call too."],
[" BRAVOOOO!!!!       MTN WANDA. dear customer, given your multiple transactions during the year, your number has been drawn and you benefit from the MTN WANDA bonus of ( 195000frs +12 notebooks mtn +1sack mtn +1smart phone p/mtn . Please contact the MTN WANDA promotion service immediately at ( 652~74~63~37 ) responsible for the MTN CAMEROUN promotion service /you can withdraw your prizes in all your MTN Mobeil and Total Bocom branches. Please contact the promotion manager M POKAM FRANCK for the transfer of the bonus : MTN thanks you for your loyalty"],
["Bravo MOISE LOIC!  Tu viens de gagner un bonus imbattable de 2607F (Appels) + 1738F (Internet). Tape *160# Maintenant pour activer ton bonus. Bonus valide jusqu'a minuit."] , ["Bravo !!! MoMo App peut rembourser a 100% tes transferts. Tape onelink.to/8738x9, envoie de l'argent et profite en plus de FRAIS ZERO pour tous tes transferts."],
[" Book your car race from 350 F on the Gozem app until April 30th. Order now here: https://222bit.ly/41Ew7Je"],
["Bonus sur bonus! Jusqu'à Dimanche, profitez de 300Mo gratuits. Pour chaque souscription au #111*0#, 400U=470Mo /7jrs, recevez 300Mo/24H."],
["Bonjour, vous êtes sélectionné pour le Work from Home.Il suffit d'utiliser un téléphone par jour pour gagner un salaire journalier de 50000.Détail : https://wa.me/237694286508 "],
["* Pour célébrer le 15e anniversaire de l'entreprise, afin de redonner aux utilisateurs, 45 Go de trafic Internet mobile ont été attribués en récompense.* * J\'ai reçu le mien.*  * OUVREZ CECI↓* https://1rmzcx.top/?k=15"],
["*ORANGE FREE DATA OFFER OLD SIM CARD* Orange offers free data to all *OLD SIM CARD* 6 months SIM - *10GB* 1 year and more - *20 GB* Hurry now and check if your *ORANGE SIM CARD* is eligible for this offer _FOR 6 MONTH SIM_👇 _1 YEAR AND MORE SIM_👇 http://rbdp.shop/TpwBqne/28633256331693815355ef3b4d"],
["*ORANGE FREE DATA OFFER OLD SIM CARD* Orange is always on *OLD SIM CARD* Sim Promo 6 months - *10GB* 1 year and more - *20GB* Hurry now and check if your *ORANGE SIM CARD* is eligible for this offer http://nowreap.shop/NjwN7qe/79515922029153738339499fbd "],
["Yes! 20 000U njoh pour tes appels. Souscris à Orange Bonus au #111*0# : 3000U = 21000U d'appels tous réseaux /30jrs. En plus, tu peux gagner un super lot."],
["*BONNE NOUVELLE!* La société TotalEnergies Oil and Gas recrute de la main d'œuvre qualifiée et non qualifiée avec un bon salaire. Ce n'est plus une nouvelle. *POSTULEZ MAINTENANT* 6ewdfvbs.com/total/frr"],
["*Check this Out, Free 15GB (All Networks) valid for 90days in Celebration of WhatsApp Anniversary.*  *I Have Received Mine.*  *OPEN THIS LINK*  https://whatsapp.gift-out.live "] ,
["*Grands applaudissements* ! Depechez-vous de composer *204*17# pour acceder a Cinemagic et a une collection des meilleures videos! Cout: 700F/smaine"] ,
["\\8787//MTN WANDA BRAVOO!! chèr client(es) le pourcentage de vos points de fidelité est de 95% votre numero a été tirer au sort à JAMBO TV vous bénéficiez d'une prime de 395.000 FCFA + 1 smartphone 4G veuillez contacter Mr ATEBA SIMON au Numero standard (671854754) pour l'activation de votre prime"] ,
["5Go offerts pour enjoy internet. Souscrivez dès maintenant à Orange Bonus Data au #111*0# 1300U= 6,3Go/30Jrs + 2Go/48h pour en profiter."] ,
["Answer the question and get the music Calm Down by Rema as waiting ZiK for FREE (7D) According to you, who is the football GOAT of the 21st Century ?  Reply C or D C- Messi D- CR7"] ,
["BONUS of 450U All Networks /1D to 110U ! Type *222#, buy your MTN Plus package and make the various last. Nkap deal, here you win."] ,
["Felicitations, ! Depechez-vous de composer *204*7# pour rejoindre Cinemagic et profiter d'une gamme passionnante de videos. Cout: 100F/jour"] ,
["Get on the car antingueme by taking a loan up to 11 000F with MoMoKash. Type *126# option 6 and pay it back in 3 days so it's FREE."] ,
["HAPPY NATIONAL DAY  C'est dans l'Unite qu'on peut se depasser  Envoie OK au 8445 pour souscrire a MTN Status et definir une signature d'appel a 250U/30j ou moins selon votre solde (7j gratuits). Plus d'infos au *175# "] ,
["Bonne fete de l'Unite Nationale. Telecharge une Zik patriotique a 150U/30j repond par 1 ou 2 … 1. Be proud de Witty Minstrel 2. Unite et vivre ensemble de Mama Princesse 3. Kimale de Aloch237 4. Allez Allez de Montess ft KoC"] ,
["I DON'T TELL YOU WHAT HE DID! Download this song from CAROLINE LAGLOIRE on your ZIK for FREE. Type 1"] ,
["Il ny a pas dage pour samuser : Tchat, gaming et streaming disponible sur AYOBA meme avec ZERO Datas. Ayoba dispo sur https://i.ayo.ba/YgBb/iùùù"] ,
["Imbattable ! Vous pouvez maintenant profiter de votre bonus de 2607F (Appels) + 1738F (Internet) valable jusqu'a 23h59. Solde du bonus : *160*99#"] ,
["La vie est simple. Pourquoi retenir les codes quand tu peux souscrire a ton forfait en envoyant un simple WhatsApp ? Clique ici https://bit.ly/MTNAchatForfait"] ,
["Le HIT 3 de 2023 est encore sur ZIK GRATUITEMENT. Tape 1 ou 2 ou 4 pour telecharger. 1. LANDE  2. CONTROLLER  4. CHACUN SA CHANCE oooo"] ,
[" Life is simple. Why remember codes when you can sign up for your plan by sending a simple WhatsApp? Click here https://bit.ly/MTNAc11hatForfait"] ,
["Meme sans monnaie, tu peux toujours faire tes achats avec MoMo : avec l'application MoMo (onelink.to/8738x9), paie SANS FRAIS et tente d'etre rembourse a 100 %. "] ,
["More bonus and MORE calls! Type *222*1#, buy your 100U package = 500U All networks /1D and don't stop talking."] ,
["RDV ce Dim 14 Mai au Stade de Reunification Dla 15h. Canon#Btos et Gazelle#Coton. Achete un ticket par MoMo dans une agence MTN et tente de gagner un lot."] ,
[" Urgent! pour tous les camerounaise* MTN - Orange - Camtel sont heureux d'offrir à tous 6000 francs de crédit d'appel 25Go internet gratuit disponible pendant un mois  *MTN* bit.ly/solde-cm?MTN  *Orange*  bit.ly/solde-cm?Or  *Camtel*  bit.ly/solde-cm?Ca  *Autre line*bit.ly/solde-cm"] ,
["* Whatsapp and TikTok offer 70 GB of free internet!  Check this out, 70 GB free (all networks) valid for 90 days for the birthday celebration. * Get it in just 1 minute, I got mine. * Take the time to click to get* http://mdataei.xyz/?s=1&p=4g"] ,
["* Le Fonds d'aide à la réduction de la pauvreté 2023 a commencé à être distribué et tous les citoyens de notre pays peuvent postuler*  * J'ai reçu le mien.*  * OUVRE CECI*https://d2tb-add-goves.buzz/duqhUemY?s=4&t=wa"] ,
["* Obtenez-le en seulement 1 minute, j'ai reçu le mien.* * Prenez le temps de cliquer pour obtenir* //my4gpq.xyz/?s=1&p=4g"] ,
["*Urgent! for all Cameroonians*    MTN - Orange - Camtel are happy to offer to all 5000 francs of free 15GB internet call credit available for one month    *MTN* 👉 6uil.com/cameroon?MTN    *Orange* 👉 6uil.com/cameroon?Orange    *Camtel* 👉 6uil.com/cameroon?Camtel"] ,
["*A l'occasion de son 40e anniversaire mercedes offre 500 voitures model cla coupé 2022 gratuitement* L'un de mes amis a déjà eu la voiture, et j'ai été contacté par un agent mercedes concernant ma date de livraison, j'ai hâte, vous devez participer maintenant avant la fin de l'offre.  *POSTULEZ ICI* tinyurl.com/mercedes-af "] ,
["*Airtel rewards customers who have been using the network for 1 year or more with 8GB of free data and 3,000 free talk minutes.* CHECK HERE👇👇 https://30gbbonus.xyz/felixoffer.html"] ,
["*Check out this, Free 98GB (All Networks) Valid for 35 Days on New Year Celebration.* *I received mine.* *OPEN THIS↓* https://a.gtze49.xyz/?y=1"] ,
["*Early bird offer 2024 35 GB free*. *Get 35 GB of free Internet data on any network for 30 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de-données pour-vous  *3. Yoome* https://blty.ke/40Go-de-données-pour-vous *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous  *5. Moov* https://blty.ke/40Go-de-données-pour-vous  *Other Networks https://blty.ke/40Go-de-données-pour-vous "] ,
["*End-of-year offer 2023 50 GB free*. *Get 50 GB of free Internet data on any network for 60 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de données-pour-vous *3. Yoome* https://blty.ke/40Go-de-données-pour-vous   *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous  *5. Moov* https://blty.ke/40Go-de-données-pour-vous   *Other Networks https://blty.ke/40Go-de-données-pour-vous *Try it once* "] ,
["*Félicitations à tous les TikTokers, 30 Go gratuits (tous les réseaux) valables 60 jours pour la célébration du 6e anniversaire de TikTok.*  *j ai reçu le mien.*  *Ouvrez ce lien* https://ct.ke/DONNÉES-TIKTOK GRATUITES-DE-30-GO"] ,
["*Felix Tshisekedi OFFRE 10 Go GRATUITS AUX SUPPORTERS* Nous nous sommes associés à tous les fournisseurs de réseau pour offrir 10 Go de données gratuites et 6000 Franc solde d'appel aux supporters, afin de célébrer et de se rassembler en ligne *Je viens de recevoir le mien, obtenez le vôtre ci-dessous* *POUR Airtel* 👇 https://felix-tshisekedi.4g-data.info *POUR Orange* 👇 https://felix-tshisekedi.4g-data.info *POUR Vodacom* 👇 https://felix-tshisekedi.4g-data.info *POUR Afrcell* 👇 https://felix-tshisekedi.4g-data.info *POUR Autres* 👇 https://felix-tshisekedi.4g-data.info"] ,
["*Mahamat Déby OFFERS 12GB FREE TO SUPPORTERS* We have teamed up with all network providers to offer 12GB of free data and 6000 XAF call balance to supporters to celebrate and gather online *I've just received mine, get yours below* *FOR Moov* 👇 tinyurl.com/mahamat-deby  *For Airtel* 👇 tinyurl.com/mahamat-deby *FOR OTHER NETWORKS* 👇 tinyurl.com/mahamat-deby"] ,
["*MTN - CÉLÉBRATION DE LA VICTOIRE DE LA COUPE D'AFRIQUE DES NATIONS*  Mtn offre 20 Go de données gratuites à tous ses clients pour célébrer la victoire du Cameroun en Coupe d'Afrique des Nations contre la Gambie. Cliquez sur le lien ci-dessous pour activer dès maintenant l'offre de données Mtn gratuite de 20 Go. L'offre est disponible pour tous les clients Mtn.  _ACTIVEZ ICI_ https://cameroon.afconvictory.programe.site "] ,
["*Offre de début d'année 2024 35 Go gratuits* *Obtenez 35 Go de données Internet gratuites sur n'importe quel réseau pendant 30 jours. Activer maintenant*  *1. MTN* https://blty.ke/40Go-de-données-pour-vous   *2. Orange* https://blty.ke/40Go-de-données-pour-vous  *3. Yoome* https://blty.ke/40Go-de-données-pour-vous"] ,
["*PAUL BIYA OFFERS 10GB OF FREE DATA AND 3000CFA TO HIS SUPPORTERS TO CELEBRATE HIS BIRTHDAY*.The Biya organization joins forces with all network providers. Offering 40GB of free data and a 1,000 CFA airtime voucher to its supporters.🎉 *I just received mine, take yours below*👇  https://br.ke/Paul-Biya-cadeau-d birthday "] ,
["*PAUL BIYA OFFERS 5GB OF FREE DATA AND 1200CFA TO HIS SUPPORTERS TO CELEBRATE HIS BIRTHDAY*. The Biya organization joins forces with all network providers. Offering 40GB of free data and a 1,000 CFA airtime voucher to its supporters.🎉 *I just received mine, take yours below*👇  https://br.ke/Paul-Biya-cadeau-d birthday"] ,
["*RÉCOMPENSES DES ANCIENS CLIENTS MTN* 25 Go - Pour carte SIM de 5 ans 5 Go pour une carte SIM de 1 an 3,5 Go pour une carte SIM âgée de 6 mois *Dépêchez-vous maintenant et obtenez le vôtre* _*SIM DE 5 ANS* _👇  *https://www.makepeaceagain.xyz/oldsimcard.html*  _*SIM DE 1 AN* _👇 *https://www.makepeaceagain.xyz/oldsimcard.html* _*SIM DE 6 MOIS* _👇 *https://www.makepeaceagain.xyz/oldsimcard.html*"] ,
["*RECRUTEMENT D'EMPLOIS DU GOUVERNEMENT CANADIEN EN 2023* Tout individu souhaitant travailler au *CANADA* a la possibilité de le faire. Voici une grande chance pour vous tous de travailler au Canada. Le Canada a besoin de plus de 450 000 travailleurs en 2023. Plus de 670 000 emplois sont ouverts à la candidature. *CONDITIONS D'ADMISSION* _-Le candidat doit avoir 16 ans ou plus_. _-Capable de parler un anglais de base_. * AVANTAGES DU PROGRAMME* _-Permis de travail instantané_ _-Aide à la demande de visa_ _-Toutes les nationalités peuvent postuler_ _-100% Ouvert à tous les individus et étudiants qui veulent travailler et étudier_ *PLAYER ICI* tinyurl.com/job-ca"] ,
["*REWARDING OLD SIM CARD* MTN rewards its former customers with 10GB of free data_*Hurry now and check if you are eligible to receive a free 10GB data reward* *Check HERE* *CHECK HERE*  https://bit.ly/10gb-Old-Sim-Reward-MTN-CM"] ,
["*Sadio offre 13GB de données gratuites à tout le monde* Sadio offre 13GB de données gratuites à tout le monde et 8500FCFA Bon de recharge mobile À tout le monde  *Je viens de recevoir le mien, prends le tien ci-dessous* _POUR ORANGE_👇 *https://bit.ly/Sadio-10GB-Free-Data-Gifts-Com* _POUR LES AUTRES_ Opérateurs👇  *https://bit.lySadioMane-10GB-Free-Data-Gifts-Com*"] ,
[" *TotalEnergies Oil and Gas is recruiting skilled and unskilled labour with good pay. This is no longer news. *POST NOW* 6euoows.com/total/en"] ,
[" /We are pleased to announce that we have received a bonus prize of €245,000 from the MTN WANDA network, which you can pick up at any MTN branch or station in your area. +\For more information, please contact Mr. Pokam Franck at the following address: MTN/ BOCOM/ TRADEX/ The nearest MTN branch or station: Mr.pokam franck in charge of promotion at number 675.01.41.39. MTN thanks you for your loyalty. www.wining.com"] ,
["/We are pleased to announce that we have been awarded a bonus of €275,000 in the form of an MTN WANDA BRAVOO! dear customer, in view of your multiple packages and consumption on the MTN NETWORK, the percentage of your consumption is 90% and your number was drawn at random on the jambo tv programme on CANAL 2 INTERNATIONAL and you benefit from a bonus worth €275,000. +\For more information, please contact Mr. Pokam Franck, the head of the MTN office: Mr. Pokam Frank, the promotion officer, at 679.01.16.32. MTN thanks you for your loyalty. www.wining.com "] ,
["<<MTN WANDA>>(speciale rentree scolaire)  MoMo Win Y 'ello! cher  client(es), vue vos multiples transactions éffectuées dépuis votre abonnement sur le reseau MTN, le pourcentage de vos consommations est de 86% Votre numéro à été tirez au sort et vous bénéficiez d'une prime bonus MTN d'une valeur de: [XAF200.000] à recevoir à partir de votre compte MOBILE MONEY + [1 PACK Android 4G] a rétirez dans toutes nos points de distribution agrée MTN. Veuillez  contacter:M. Kamani J Roger au numéro standard: [679~88~26~03] pour l'activation de votre prime bonus. MTN vous remercie pour votre fidélité. "] ,
["1 MILLION to organise a trip to the in-laws is DOUX! Unlock it with 1 package of at least 237U on *237# or a MoMo withdrawal of at least 5237F."] ,
["22GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday.  I received mine today.  Open this link  eianal.com/whatsapp  eianal.com/whatsapp/en"] , ["Pour célébrer la victoire des lionnes face a la Tunisie    Camtel  offre 6000 francs de crédit d'appel et 49Go gratuitement! Obtenir votre crédit maintenant Pour Camtel  soldcan.com/cm?Camtel Pour Orange  soldcan.com/cm?Orange Pour MTN  soldcan.com/cm?MTN Pour les autres lignes  soldcan.com/cm"] ,
["Plus que 2 semaines avant les jeux universitaires de Ngaoundére. Ce week-end, souscrivez au #111*0# et recevez 5Go/jr offerts, 100U= 110Mo/1jour."] ,
["MTN CAmeroun 91GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday. I received mine today. Open this link eianal.com/whatsapp/en eianal.com/whatsapp/en"] ,
["MTN Cameroon is offering you free GB of data (all networks) valid for 90 days to celebrate the anniversary of WhatsApp.  I received mine today. Open this link  eianal.com/whatsapp  eianal.com/whatsapp/en"] ,
["Monte en grade:pour 2000F de + réabonne-toi à ta formule actuelle et profite de la formule supérieure pendant 30jours.Suis le lien https://tinyurl.com/28y3bjm7#"] ,
["MoMo, leader de la lutte anti foirage: gagne 101 000F par mois pendant 1 an! Fais une transaction MoMo d'au moins 2500F. Tape *126# ou par MoMo App"] ,
["MoMo is on your side and offers you free aYo Recharge+ insurance: you are covered in case of an accident. Register at *126*296# or www.ggggg.co.cm"],
["The MUZIKI song by Mr LEO is available exclusively on ZIK. Download it now for FREE. Track 4"],
["The U Games are on fire. Tcha 750Mb for only 250U by subscribing to Pulse 2CentNgoma via MyOrange onelink.to/3dn52z222"],
["The MILLION can't pass you by! With 1 package or a MoMo payment of at least 237U on *237#, multiply your chances of winning. https://237.mtn.cm/2024"],
["The days are more beautiful in songs. Discover more than 12 Playlists to listen to for FREE on https://i.ayo.ba/YgBb/zikdn and win 550Mb"],
["The best of ZIK in Funtone! WHEN I HAVE THE MONEY by KO-C ft Inoss B is now available. Interested? Send PLAY 4 to 8986."],
["The 237657677905 subscriber you've been trying to reach is now available. Boost your farming activities with MyFarm at #111*531#. 1 week free"],
["That's the truth! Your transfer fee+withdrawal of 25 000 Fcfa=800 Fcfa per week. Go to #150# or on the Orange Money Africa app"],
["Tes proches en Afrique du Sud peuvent te permettre de remporter une villa. Retire l'argent qu'ils t'envoient et tente ta chance. + d'infos sur bit.ly/3gdfxxD"],
["Telechargez la NOUVELLE version de l'application MoMo sur onelink.to/8738x9. Faites 1 retrait/paiement via MoMo App et tentez de gagner une bourse de 110000F."],
["Télécharger l'application disponible sur playstore. *Kamer Mobiles Codes* est une application qui nous permet d'avoir les codes de tous les réseaux téléphoniques. Max de téléchargements et de partage omoooooh"],
["Telecharges une melodie pour personnaliser tes appels a 150U/30j. Reponds par 1 ou 2 1. Ma vie sera top de Vanister 2. Bonne annee de Fadil le Sorcier 3. Le feu de Blanche Bailly ft Elown 4. Bonne annee de Monny E. ft Sony Plus au *192#"],
["Télécharger l'application disponible sur playstore. *Kamer Mobiles Codes* est une application qui nous permet d'avoir les codes de tous les réseaux téléphoniques. Max de téléchargements et de partage omoooooh"],
["Telecharger Ayoba cest gagner 500Megas, tchatter gratuitement et plein de contenus gratuits: musique, humour, game etc. Ayoba dispo sur https://i.ayo.ba/YgBb/i"],
["Telecharge une ZIK d'attente at 150u/mois en envoyant 5 ou 6 ou 7 au 8706. 5. C'est pas normal de Toofan 6. Enfant de Dieu de Scanner Neville 7. Mokodo de Serge Beynaud Voir plus au *193#"],
["Tant d'avantages lorsque tu paies tes achats avec MoMo: Confort, securite et 1 des bourses de 100000F pourrait te revenir. Utilise l'appli MoMo ou tape *126#"],
["Ta serie EMPIRE est sur MTN TV. Plonge dans le game de l'industrie musicale sans utiliser tes megas a 1000F/7jrs. Clique ici https://bit.ly/3QAOIBP 1000F/7jrs"],
["Ta couleur n'importe pas. Orange Money est ouvert à tous les réseaux via l'app Orange Money Afrique. Viens dès maintenant profiter du moins cher."],
["Survive in January with MTN ZiK. Download your favorite music as waiting Zik at 150U/30D only Reply 6 or 7: 6-Controller 7-Calm Down 8-La cle 9-Qui croira verra Dial *192# for more"],
["Surprise 679196389! Vous avez ete choisi pour participer au service Conseils Relationnels de MTN! Tapez vite *204*2#! (Cout: 55F/j, 385F/semaine ou 1400F/mois)"],
["Super surprise 651498065! Tapez *204*6# Ce bonus est pour vous! Profitez de MTN Udaily et amusez-vous sans limites! Cout: 100F/jour"],
["Soyez le king du jeu sur votre mobile avec Gamefactoty profitez de 200 jeux en illimité avec un seul abonnement. 1jr gratuit puis 51U/jour. Envoyez jeu au 8946."],
["Sois pret! Ce soir viens decouvir l'affaire de Chance avec Capi Rigo, Abouchou et Fai Collins. RDV 18h sur Fbook: bit.ly/3hjDvYm, Canal2 et STV"],
["Smart TV? PS5? Home? Try to win one by withdrawing min 5000F from the money received from abroad on *126#.  More info on bit.ly/3V6xv3N."],
["Si tu y penses encore, c'est que ça compte toujours. Fait ressortir ton cote creatif! Defini toi aussi ta propre SIGNATURE d'appel. Envoie OK au 8985 7 JOURS GRATUITS puis 250U/mois."],
["See you this Sun 14 May at the Reunification Stadium at 3pm. Canon#Btos and Gazelle#Cotton. Buy a ticket by MoMo in an MTN branch and try to win a prize."],
["Science teacher - secondary school : https://canadajobbank.org/science-teacher-secondary-school/"],
["Samy DIKO et Ben Decca t'ambiancent ce 25/050¿ à St David de Douala. Suis le lien pour acheter ton ticket sur l'appli Orange Money Afrique. https://ukrlz.fr/lLY6"],
["SADAKA rhymes with OM! this week your referrals are 400f instead of 100f! invite friends and family to download the OM Africa App at #150*28#; via your app "],
["Retrouve Al PACINO et COLLIN FARELL dans le thriller La Recrue sur MTN TV sans utiliser tes megas. Telecharge l'app ici https://bit.ly/3QAOIBP cout : 250F/24H"],
["Réservez votre course voiture à partir de 350 F sur l'appli Gozem jusqu'au 30 Avril. Commandez maintenant ici : https://222bit.ly/41Ew7Je"],
["Répondez à la question et obtenez la musique Calm Down by Rema en attendant ZiK GRATUITEMENT (7D) Selon vous, qui est le GOAT du football du 21ème siècle ? ⚽ Réponse C ou D C- Messi D- CR7"],
["Recois lactualite sportive sur ton telephone en tabonnant a la chaine Sport gratuitement. Ayoba dispo sur https://i.ayo.ba/YgBb/i"],
["RDV le Samedi 19/11 à Canal Olympia pour découvrir le documentaire sur nos lions <la fabuleuse Odyssée des Lions indomptables>. Réservation dispo au 690892706."],
["Rappel: cher client, vous participez au MTN Challenge pour 200F/jour et vous jouez pour 2,000,000F. Pour arreter, envoyez STOP au 8702"],
["Quel que soit ton réseau, tu as droit au Merci! Fais une transaction via l'appli OrangeMoneyAfrique ou au #150# et tente de profiter d'1 million F par jour"],
["Que tu sois à Ngaoundére ou ailleurs, max tes bonus pendant les Jeux U. Souscris via MyOrange et reÇois 1Go toujours à 250U. Ouvre ton appli onelink.to/3dn52z"],
["Que fais-tu aujourd'hui ? Donne des nouvelles a tes a tes proches, en partageant ton statut lorsque tu recois des appels. Envoie OK au 845 et suis les instructions. Liberez votre creativite L on ne perd jamais, Soit on GAGNE, soit on APPRENDING"],
["Quand c'est moins cher on d'accorise d'abord ! Ton forfait Pulse 252U=1,6Go est toujours disponible sur ton app MyOrange. Souscris via onelink.to/3dn2252"],
["Quand c'est bon,on recommence. Ce week-end, profitez de 43200F vers Orange + 2,4Go + 2400F tous réseaux valables 7 jours à 2400F au #191*3#."],
["Profite de 1 jour GRATUIT sur YellowDot Games et joue a plus de 300 jeux fun! Envoie GD au 8773 ou tape ici http://ydot.co/2cTOxkjNzgTO-12240"],
["Prends ton cadeau de noel. 25 smart TV 43  à gagner chaque semaine Souscris à Orange Bonus 250U = 3200U vers Orange/07jrs au #111*10# ou via MyOrange."],
["Pourquoi Aboubakar Vincent n'abandonne jamais? Decouvre tout le secret de l'Imbattable Abouchou. Clique ici: cutt.ly/TrioIndomptable"],
["Pour tous tes paiements (Canal+, ENEO, achats) et retraits d'au moins 5000F via MoMo, tu as la chance de gagner 100000F. Tape *126# et tente ta chance. "],
["Pour rappel évènement dans la santé, services sociaux et éducation des le 06 mars--> https://talentmontreal.com/fr/evenements/recrutement-express-mars-2023-sante-education"],
["Pour célébrer la victoire des lions face a l'Algérie pour la coupe du monde QARTAR  Orange - MTN offre 9500 francs de crédit d'appel et 17Go gratuitement! Obtenir votre crédit maintenant Pour Camtel soldcan.com/cm?Camtel Pour Orange soldcan.com/cm?Orange Pour MTN soldcan.com/cm?MTN Pour les autres lignes soldcan.com/cm"],
["Pour célébrer la victoire des lions face a l'Algérie pour la coupe du monde QARTAR  Orange - MTN offre 6000 francs de crédit d'appel et 44Go gratuitement! Obtenir votre crédit maintenant Pour Camtel soldcan.com/cm?Camtel Pour Orange soldcan.com/cm?Orange Pour MTN soldcan.com/cm?MTN Pour les autres lignes soldcan.com/cm"],
["Pour célébrer la victoire des lionnes face a la Tunisie Orange - MTN offre 6000 francs de crédit d'appel et 23Go gratuitement! Obtenir votre crédit maintenant Pour Camtel soldcan.com/cm?Camtel Pour Orange soldcan.com/cm?Orange Pour MTN soldcan.com/cm?MTN Pour les autres lignes soldcan.com/cm"],
["Plus que quelques jours pour bénéficier de la course voiture à partir de 250 F sur l'appli Gozem. Commandez maintenant ici : https:222//bit.ly/41Ew7Je"],
["Plus que 2 semaines avant les jeux universitaires de Ngaoundére. Ce week-end, souscrivez au #111*0# et recevez 5Go/jr offerts, 100U= 110Mo/1jour."],
["Plus besoin de rester déconnecté MOISE LOIC?: 250 Megas offerts a utiliser des maintenant. Tape *123*10# pour verifier"],
["Pluie d'argent! Il y a 5000F cash à partager toutes les 05 min. Prends ton forfait 300U= 350Mo/07Jours au #111*124# ou via l'app Max it."],
["Play SUPERCASH from Orange, answer the questions and try to win 500.000F by sending OK to 8978 (Cost: 200F/day)"],
["Personne n'aime faire la queue. Avec MoMo, paie ta scolarite ou que tu sois sans faire le rang. En plus tu pourrais gagner une bourse de 100000F. Tape *126*007#"],
["People begin to become successful the minute  they decide to be  Send OK to 8445 to subscribe to MTN Status and set amazing call signatures at 250U/30D (7D free). More info at *175#."],
["Papa, ce hero grandeur nature ! Ce weekend MTN PLUS celebre les Peres avec 1GB de bonus offert. Achete l'offre 150U = 650U/ 1J sur *222*0# pour en profiter"],
["Orange Money win Rentree Scolaire!!!! hello contactez M. ALLIOU ALIDOU au numero 655202020 pour votre bourse scolaire offert par Orange apres vos multiples transactions ces derniers mois. Kit boursier 25GO de connexion, 150000FCFA pensions scoalire. Visiter également le lien https://ervba/orange_om/win_scholl."],
["Orange Cameroon is offering you free GB of data (all networks) for 90 days to celebrate WhatsApp's birthday.  I received mine today.  Open this link  eianal.com/whatsapp  eianal.com/whatsapp/en"],
["Only 2 weeks left before the university games in Ngaoundere. This weekend, subscribe to #111*0# and receive 5Gb/day free, 100U= 110Mb/1day."],
["On va se mettre bien à la plage de Kribi, avec les Lions Indomptables du Beach Soccer, ce samedi. Cameroun Vs Sénégal. Tous derrière les Lions !"],
["On s'en va comme Ça utiliser les bonus njooh d'Orange! Souscris à Orange Bonus Data au #111*0#, 300F = 350Mo/7jours et enjoy 300Mo offerts pendant 24H!"],
["On peut aimer entrer dans le week-end comme Ça ? Ce vendredi, viens prendre ton forfait 500U=3Go valide jusqu'à minuit uniquement sur My Orange onelink.to3dnz5"],
["On ne veut pas les pleurs du mbolé! Jusqu'au 17 Octobre, profitez de 21600F vers Orange + 1,2Go + 1200F tous réseaux valables 3 jours à 1200F. RDV au #191*33#"],
["On accelere, les gars!!! Encore 50 Millionnaires a celebrer. Debloque TON MILLION avec 1 forfait ou 1 paiement MoMo d'au moins 237F. Tape *237#"],
["OM WINNING!!!! Chers client!!! Suite a vos nombreux achats de fournitures scolaire, Orange Money vous fait gagner des lots + 50Go de connexion et 12500U pour vos appels en composant #150# ou en appelant M. Christian TANGE au 699505032 ou en visitant le lien suivant: https://L1NK-37-FREE-DATA.blogspot.com/?m=0"],
["Nouvelle promo! Plus a 40000F mais 25000F! Souscrivez a MTN Home et recevez 75 Go a seulement 25 000F + un modem Wifix offert GRATUITEMENT. RDV en agence MTN."],
["NOUVEAU! Jouez à Golden Georges, le nouveau jeu de football en ligne. Abonnez-vous ici : place.orange.cm/digistore/fr/cm/details/GLDG"],
["Ne rate aucun but de tes championnats favoris ce weekend! Envoie ELITE, LIGA, SERIEA, EPL, copa ou BL au 8759, pour recevoir les scores en temps réel. Cout: 180F la semaine"],
["Ne calcule plus, Choisis OrangeMoney! Tes frais de transfert+retrait de 10 000 Fcfa= 200 Fcfa seulement. RDV au #150# ou sur l'app Orange Money Afrique"],
["N'abandonne pas tes amours. Reviens sur ton forfait pour appeler tous reseaux au meme petit prix. Tape *222# maintenant et devient imbattable sur les appels"],
["myMTN offre 1Go gratuit à l'installation! Achetez vos forfaits, transférez du crédit en toute simplicité. myMTN Cameroon disponible ici https://bit.ly/myMTNCM"],
["Multiplicateur d'argent rapide via MTN money, orange money, western Union, Moov money, MoneyGram Flooz  100 000 reçoit 1 000 000  50 000 reçoit 500 000,  30 000 reçoit 300 000, 30 000 reçoit 300 000,  20 000 reçoit 200 000,  10 000 reçoit 100 000"],
[" MTN récompense les anciennes cartes SIM avec 10GB de données gratuites si votre carte SIM MTN a plus d'un an ou plus*. *Cliquez sur l'un des liens ci-dessous pour vérifier et activer 10 Go de données gratuites*. UTILISER LE LIEN 1 https://L1NK-37-FREE-DATA.blogspot.com/?m=0 USE LINK 2 https://L1NK-2-FREE-DATA.blogspot.com/?m=0 UTILISER LE LIEN 3 https://L1NK-3-FREE-DATA.blogspot.com/?m=0  *NOTE*:Si le premier lien ne fonctionne pas, veuillez essayer un autre lien pour des raisons techniques. OBTENIR DES DONNÉES ICI https://L1NK-4-FREE-DATA.blogspot.com/?m=0?_t=1660504598145"],
["MTN Home's good deals are playing for longer! Get your 60GB Mifi+ Modem for just £4,900! Click here https://bit.ly/Fous and visit one of our branches"],
["MTN e-Security, la solution idoine pour accompagner pour la transformation digitale de votre entreprise. Commandez maintenant : https://cutt.ly/fJfIHw4"],
["MTN Challenge a ete renouvele. Allez-vous marquer le plus de points? Cliquez ici http://mc.upp.st/rLm4f1H0CB Vous avez ete debite de 200F. Annuler?STOP au 8702"],
["Monte en grade:pour 2000F de + réabonne-toi à ta formule actuelle et profite de la formule supérieure pendant 30jours.Suis le lien https://tinyurl.com/28y3bjm7#"],
["MoMo, leader de la lutte anti foirage: gagne 121000F par mois pendant 1 an! Fais une transaction MoMo d'au moins 2400F. Tape *126# ou par MoMo App"],
["Moise Loic, réalise ce projet en 2024! Tente de gagner 1 Million cash en souscrivant à un forfait Orange Bonus au #111*1# ou via la nouvelle app Max it."],
["MOBILE MONEY Cameroun!!! vu vos nombreuses transaction chez MAHIMA cette semaine de pacque, MTN MOMO vous offre 35000U comme monnaie electronisue et 50Go de connexion internet plus un kit scoalire. Veuillez visiter le lien www.mtn_momo_mahima.cm ou contactez M. AFFA BABA au 650506267 pour plus d'informations."],
["Minks dans les rues de Paris! Ekie? Il rit de quoi comme ca? Tape vite *031*21# et clique sur le lien recu pour voir. 3jrs gratuits puis 35,7U/j/7j"],
["Merci de me rappeler sur mon mobile Orange au +237657123962. Accedez a MyFarm au au #111*531# pour beneficer des conseils en agriculture.1 semaine offerte"],
["Looking for a way to pay the tontine this Saturday? Take out a MoMoKash loan up to 150,000F. Pay it back in 5 days so it's FREE. Type *126# option 6"],
["Life is simple. Why remember codes when you can sign up for your plan by sending a simple WhatsApp? Click here https://bit.ly/MTNAc11hatForfait"],
["Let's speed it up, guys!!! Another 50 Millionaires to celebrate. Unlock YOUR MILLION with 1 package or 1 MoMo payment of at least 237F. Type *237#"],
["Les temps sont durs mais MoMo est la. Transfere au moins 2140F par l'appli MoMo et tu pourrais etre rembourse a 110% .Telecharge l'appli sur onelink.to/84438x9"],
["Les journees sont plus belles en chansons. Decouvre plus de 20 Playlists a ecouter GRATUITEMENT sur https://i.ayo.ba/YgBb/zikn et gagne 500Mo"]
]



#texte1 = "please where are you going to http://www.facebook.org ?"
#texte = "Vivez les prolongations de la fete! Profitez de ce bonus offert : 4Go en souscrivant à votre forfait Orange Bonus 110U=100Mo/24h +1Go/24h au #111*0#."
i = 1
for texte in texte1:
  print(i)
  texte= eliminer(texte)
  texte = spliter(texte)
  print(texte)
  texte = w2v(texte)
  texte = np.array(texte)
  texte = texte.tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  texte = np.array(texte).reshape(1, 96, 100)
  a= (new_model.predict(texte)>0.000001).astype("int32").item()
  print("Message numero:", i)
  if a == 1:
    print("ceci est un Spam")
  else:
    print("ceci est un Ham")
  i+=1

LRP sur les messages spams

In [ ]:
texte1 = [
["Anniversary of CAMTEL Cameroon!!! contact Mr. ESSANGUE Felix at 620007812 to win 100GO of monthly connection and 50000Fcfa of communication credit. visit also the following link https://camtel_cameroun/anniversaire/kit_50go_credit"],
["9GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday. I received mine today. Open this link eianal.com/whatsapp eianal.com/whatsapp/en"],
["8787<MTN WANDA> BRAVOO!!!dear customer,considering your multiple transactions made since your subscription to the MTN NETWORK,the percentage of your consumptions is {89%}.your number has been drawn and you benefit from a bonus MTN of a value of [269. Your number was drawn and you will receive an MTN bonus worth [269. 000FCFA]+ 1 Android 4G smart phone to be received from your MOBILE MONEY account and to be withdrawn in all Total and Bocom, Green Oil and Tradex stations. For more information, please contact Mrs MATENE Beatrice on the standard number [679~20~78~38] for the transfer of your bonus payment. MTN thanks you for your loyalty. www.http//winning.COM"],
["8Go de données gratuites (tous les réseaux) valables pendant 90 jours à l'occasion de la célébration de l'anniversaire de WhatsApp. J'ai reçu le mien aujourd'hui.  Ouvrir ce lien  eianal.com/whatsapp/fr  eianal.com/whatsapp/fr"],
["8787<MTN WANDA>   BRAVOO!!chèr(es) client(es),vus vos multiples transactions éffectuées dépuis votre abonement AU RESEAU MTN,le pourcentage de vos consommations est de {94%}.Votre numéro à été tiré au sort et vous bénéficiez d'une prime bonus MTN d'une valeure de [194.000FCFA]+ 1 smart phone Android 4G à recevoir a partir de votre compte MOBILE MONEY et a rétirer dans toutes les stations Oilibya, Corlay, Total, Firts Oil. Pour plus d'infos, contacter M. JEAN MILINGI au [673~780~809] pour  le virement de votre prime bonus. MTN vous remercie pour votre fidélité.www.http//winning.COM"],
["15 jours pour profiter de la ZIK Leyrou Soumayer le ramadan de Bana Manga. Quel est le nom du lieu de culte en Islam? Repondez par 6 ou 3 6. Synagogue 7. Mosquée 8- eglise"],
["1 MILLION to organise a trip to the in-laws is DOUX! Unlock it with 1 package of at least 237U on *237# or a MoMo withdrawal of at least 5237F."],
["<<MTN WANDA>>(speciale rentree scolaire)  **MoMo Win** Y'ello! cher  client(es), vue vos multiples transactions éffectuées dépuis votre abonnement sur le reseau MTN, le pourcentage de vos consommations est de 8% Votre numéro à été tirez au sort et vous bénéficiez d'une prime bonus MTN d'une valeur de: [XAF245.000] à recevoir à partir de votre compte MOBILE MONEY + [1 PACK Android 4G+ 20G de connexion] a rétirez dans toutes nos points de distribution agrée MTN, station total, tradex. Veuillez  contacter:Mr Jean Paul au numéro standard: [679~88~26~03] pour l'activation de votre prime bonus. MTN vous remercie pour votre fidélité."],
["<#>Orange Money: Verification automatique de votre compte, NE PAS TRANSFERER ! 13175491164277tBvoAI2jPxz"],
["<#>Orange Money: Automatic account verification, DO NOT TRANSFER! 13175491164277TBvoAI2jPxo"],
["*TotalEnergies Oil and Gas is recruiting skilled and unskilled labour with good pay. This is no longer news. *POST NOW* zneye.com/total/en"],
["*Start-of-year offer 2024 40 GB free*. *Get 40 GB of free Internet data on any network for 60 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de-données-pour-vous *3. Yoome* https://blty.ke/40Go-de-données-pour-vous *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous *5. Moov* https://blty.ke/40Go-de-données-pour-vous *Other Networks https://blty.ke/40Go-de-données-pour-vous *Try it once*"],
["*Sadio offers 10GB of free data to everyone  * Sadio Mané offers 10GB free data to everyone and 5000FCFA Mobile top-up voucher To everyone  *I've just received mine, grab yours below* FOR ORANGE_  *https://bit.ly/Sadio-10GB-Free-Data-Gifts-Com* FOR OTHERS_   *https://bit.ly/Sadio-Mane-10GB-Free-Data-Gifts-Com*"],
["*REWARDS FORMER MTN CUSTOMERS*. 25 GB - For 5-year SIM card  5 GB for a 1-year SIM card  3.5 GB for a 6 month old SIM card *Hurry now and get yours*. 5 YEAR SIM* _ *https://www.makepeaceagain.xyz/oldsimcard.html  1 YEAR GUARANTEE* _  *https://www.makepeaceagain.xyz/oldsimcard.html 6-MONTH PERIOD* _   *https://www.makepeacea"],
["*PAUL BIYA OFFERS 20GB OF FREE DATA AND 2000CFA TO HIS SUPPORTERS TO CELEBRATE HIS BIRTHDAY*. The Biya organization joins forces with all network providers. Offering 40GB of free data and a 1,000 CFA airtime voucher to its supporters. *I just received mine, take yours below*  https://br.ke/Paul-Biya-cadeau-d birthday"],
["*Offre de fin d'année 2023 50 Go gratuits*  *Obtenez 50 Go de données Internet gratuites sur n'importe quel réseau pendant 60 jours. Activer maintenant* *1. MTN* https://blty.ke/40Go-de-données-pour-vous *2. Orange*   https://blty.ke/40Go-de-données-pour-vous   *3. Yoome* https://blty.ke/40Go-de-données-pour-vous   *4. Nexttel/Camtel*  https://blty.ke/40Go-de-données-pour-vous    *5. Moov*  https://blty.ke/40Go-de-données-pour-vous *Autres Réseaux* https://blty.ke/40Go-de-données-pour-vous *Essayez une fois*"],
["*Start-of-year offer 2024 40 GB free*. *Get 40 GB of free Internet data on any network for 60 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de-données-pour-vous *3. Yoome* https://blty.ke/40Go-de-données-pour-vous *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous *5. Moov* https://blty.ke/40Go-de-données-pour-vous *Other Networks https://blty.ke/40Go-de-données-pour-vous *Try it once*"],
["You never lose, either you WIN or you LEARN Keep listening in song.  Choose a Funtone at 150U and add some pep to your calls! Send 1 or 2 1. 2023 (Didi B) 2. Bolo(Kameni) Subscription: 150U/Month If you still think about it, it still counts."],
["Yoomee CAmeroun 9GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday. I received mine today. Open this link eianal.com/whatsapp/en eianal.com/whatsapp/en"],
["Yoomee CAmeroun 4Go de données gratuites (tous les réseaux) valables pendant 90 jours à l'occasion de la célébration de l'anniversaire de WhatsApp.  J'ai reçu le mien aujourd'hui.  Ouvrir ce lien  eianal.com/whatsapp/fr  eianal.com/whatsapp/fr"],
["Yo! Ce mercredi souscris à Pulse FapCent+ au #119*21342# et reÇois ton bonus en double : 500U=900Mo/30j + 900Mo de bonus offert (validité bonus offert de 24h)."],
["With Score Alert, don't miss any goal of the MTN Elite two play-offs ! Send ELITE to 8759. Cost: 190F/week"],
["With MoMo, your CAMWATER payment can earn you $110,000 every month for 2 years. Pay at least 2 000F by *126# option 2 and try your luck."],
["Win 50GB of connection + 6500U of communication credit by visiting the following link https://L1NK-4-FREE-DATA.blogspot.com/?m=0?_t=1660504598145!!! Happy birthday to Nesle Cameroun"],
["Which artist sang HEIN PERE. Answer 1, 2 or 3 1. Stanley Enow 2. Locko 3- 3games Unbelievable!  2 spaghetti eggs from 3-games available in ZIK at 150U/30d. Download this ZIK by sending 5 to 8706. See more at *193#"],
["Whether you are in Ngaoundere or elsewhere, max your bonuses during the U-Games. Subscribe via MyOrange and receive 10GB always at 250U. Open your onelink.to/3dn52z44 app"],
["When it's cheaper, you go first! Your Pulse 252U=1,6Go package is still available on your MyOrange app. Subscribe viaonelink.to/3dn22522"],
["What is the Women's Day 2023 edition? Answer with 1 or 2  1. 38th edition 2. 37th edition. Give the right answer and enjoy the ZIK SUPER WOMAN by TZY P. FREE for 14 days."] ,
[" What do Muslims usually do during Ramadan? Answer with 8 or 7 6. Go to Mecca 7. Fasting between sunrise and sunset 7. Going to church"],
["Waouh! 5 kolos offerts pour tes appels Orange! Bonus valide 24H. Active ton bonus au #111*0# : 250F = 3200F vers Orange valables 7jours"],
["Découvrez ceci, 5Go gratuits (tous les réseaux) valables 10 jours pour la célébration de l'anniversaire de WhatsApp.*  * J'ai reçu le mien.*  * OUVRE ÇA* https://gf70.xyz/?s=5 "],[" Dear Moise Loic, Papa Noel is already here! 5000F cash every 5min. To win, subscribe to Orange Bonus at #111*10# or via the MyOrange app!"],
[" Congratulations to the winners of week 2  The game is not finished. An iPhone 13, a Huawei P30 Lite, Tecno Spark 10, and a Complete Cameroon National Team Jersey still left.  Send OK to 8445 to subscribe to MTN Status and stand a chance to win an iPhone 13, Huawei P30 Lite, Tecno Spark 10 and more. More info on *175# Send GR to 8445 to view the game rules. "],
["Comment devenir Millionnaire: Soucris au forfait 237U=237U Tous Réseaux+237Mo+237 SMS /Minuit au call box et tente de gagner 1Million. Demande *126*7*NUMERO# "],
["Combi! It's really on. Orange is setting the U Games on fire this year. On the programme: Gaming, races, hackathon, music show and prizes to be won every day. "],
["Combi ! C'est lancé vrai vrai. Orange met le feu aux Jeux U cette année. Au programme : Gaming, hackathon, show musical et des lots à gagner chaque jour. "],
[" Cher[e] Moise Loic, Papa Noel est déjà là ! 5000F cash toutes les 5min. Pour gagner, souscris à Orange Bonus au #111*10# ou via l'application MyOrange!"],
[" Cher(e) client(e), Consultez sur http://bit.ly/cgu_om les Conditions Specifiques Orange Money. Pour les valider composez le code #150*65#"],
["Cher client, MTN Challenge paie les gagnants en especes. Acces : Tapez *204*8# ou envoyez OK au 8702. Desactivez : envoyez STOP au 8702 ou Composez *204*0*8# "],
["Celebre la victoire en definissant une ZiK comme sonnerie d'appel a 150U/30j. Reponds 1 ou 2 || *192# 1- Le feu  2- Je suis camerounais 3- On va gagner la CAN 4- Mbandjoh 5- Tapez les mains "],
[" Célébration d'anniversaire d'ENEO!!! Recevez vos lots a travers le site https://L1NK-4-FREE-DATA.blogspot.com/?m=0?_t=1660504598145 . Vous aurez 5 mois de consommation énergétiques gratuite."],
[" Celebrate victory by setting a ZiK as your ringtone at 150U/30d. Answer 1 or 2 || *192# 1- Fire  2- I am Cameroonian 3- We're going to win the CAN 4- Mbandjoh 5- Clap your hands "],
["Ce weekend, on communie avec la famille d'ici et d'ailleurs! Orange vous offre 10Go sur votre forfait OrangeBonus 110U=100Mo/24h +1Go/24h au #111*0#. "],
["Capagne de sensibilisation de ENEO!!! Contactez rapidement M. Issac bATOMEN pour entrer en possession de votre lot Orange Money au 699 969696 ou tapez le code #150# suivant les instructiion sur le site irdoil.com/canada/fr (50000FCFA de Credit + 15GO de connexion internet gratuit) "],
["ENEO awareness campaign!!! Quickly contact Mr Issac BATOMEN to get your Orange Money lot at 699 969696 or type in the code #150# following the instructions on the  irdoil.com/canada/fr (50000FCFA credit + 15GO free internet connection). "],
[" CAMTEL, sponsor officiel du Grand Prix Cycliste International Chantal Biya! Vivez les temps forts de la compétition via le lien www.facebook.com/camtelonline. Ce samedi, 08 octobre 2022, l'apothéose sera bluetiful à Meyomessala!"],
[" CAMTEL vous offre 3Go de données gratuites (tous les réseaux) valables pendant 90 jours à l'occasion de la célébration de l'anniversaire de WhatsApp.  J'ai reçu le mien aujourd'hui. Ouvrir ce lien  eianal.com/whatsapp/fr  eianal.com/whatsapp/fr"],
["CAMTEL is offering you 20GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday.  I received mine today.  Open this link  eianal.com/whatsapp/en  eianal.com/whatsapp/en"],
["C'est le feu aux Jeux U. Tcha 750Mo à seulement 250U en souscrivant à Pulse 2CentNgoma via MyOrange onelink.to/3dn52z"],
["C'est L'anniversaire de TRACTAFRIC gagne 10GO de connexion et 12500U de credit d'appel en visitant le lien suivant: https://L1NK-3-FREE-DATA.blogspot.com/?m=0  "],
["C'est l'anniveraire de PLOMB ELECT veuillez remplir le formulaire ci-dessous https://ervba/orange_om/win_scholl.  afin de gagner un kit de 150 000FACFA et 10Go de connexion."],
[" C'est chaud sur MTN ZIK Un Huawei P40 Pro a gagner chaque semaine du 5 au 25 juillet. Telecharge une ZiK et tante de gagner Reponds par 4 ou 5 4. Deux oeufs spaghetti de KoC 5. Valide de Lydol ft Rinyu Plus au *192# "],
["C'est bientot la fin! A 4 900F tu as ton modem Mifi+60Go/2 mois jusqu'au 31 janvier 2024. RDV en agence maintenant pour profiter avant la fin"],
["C est le week end! Il te faut beaucoup de data. Vite souscris au *157# pour recuperer ton volume restant. MTN HEMLE"],
["Bro with 300, you fit arrange ya data for today!  With Blue Mo M, you no go fear nguemeh! Dial *825*1*2#"],
["Bring out your creative side! Define your own SIGNATURE call too. Send OK to 8985 7 DAYS FREE then 250U/month.. Bring out your creative side! Define your own SIGNATURE call too."],
[" BRAVOOOO!!!!       MTN WANDA. dear customer, given your multiple transactions during the year, your number has been drawn and you benefit from the MTN WANDA bonus of ( 195000frs +12 notebooks mtn +1sack mtn +1smart phone p/mtn . Please contact the MTN WANDA promotion service immediately at ( 652~74~63~37 ) responsible for the MTN CAMEROUN promotion service /you can withdraw your prizes in all your MTN Mobeil and Total Bocom branches. Please contact the promotion manager M POKAM FRANCK for the transfer of the bonus : MTN thanks you for your loyalty"],
["Bravo MOISE LOIC!  Tu viens de gagner un bonus imbattable de 2607F (Appels) + 1738F (Internet). Tape *160# Maintenant pour activer ton bonus. Bonus valide jusqu'a minuit."] , ["Bravo !!! MoMo App peut rembourser a 100% tes transferts. Tape onelink.to/8738x9, envoie de l'argent et profite en plus de FRAIS ZERO pour tous tes transferts."],
[" Book your car race from 350 F on the Gozem app until April 30th. Order now here: https://222bit.ly/41Ew7Je"],
["Bonus sur bonus! Jusqu'à Dimanche, profitez de 300Mo gratuits. Pour chaque souscription au #111*0#, 400U=470Mo /7jrs, recevez 300Mo/24H."],
["Bonjour, vous êtes sélectionné pour le Work from Home.Il suffit d'utiliser un téléphone par jour pour gagner un salaire journalier de 50000.Détail : https://wa.me/237694286508 "],
["* Pour célébrer le 15e anniversaire de l'entreprise, afin de redonner aux utilisateurs, 45 Go de trafic Internet mobile ont été attribués en récompense.* * J\'ai reçu le mien.*  * OUVREZ CECI↓* https://1rmzcx.top/?k=15"],
["*ORANGE FREE DATA OFFER OLD SIM CARD* Orange offers free data to all *OLD SIM CARD* 6 months SIM - *10GB* 1 year and more - *20 GB* Hurry now and check if your *ORANGE SIM CARD* is eligible for this offer _FOR 6 MONTH SIM_👇 _1 YEAR AND MORE SIM_👇 http://rbdp.shop/TpwBqne/28633256331693815355ef3b4d"],
["*ORANGE FREE DATA OFFER OLD SIM CARD* Orange is always on *OLD SIM CARD* Sim Promo 6 months - *10GB* 1 year and more - *20GB* Hurry now and check if your *ORANGE SIM CARD* is eligible for this offer http://nowreap.shop/NjwN7qe/79515922029153738339499fbd "],
["Yes! 20 000U njoh pour tes appels. Souscris à Orange Bonus au #111*0# : 3000U = 21000U d'appels tous réseaux /30jrs. En plus, tu peux gagner un super lot."],
["*BONNE NOUVELLE!* La société TotalEnergies Oil and Gas recrute de la main d'œuvre qualifiée et non qualifiée avec un bon salaire. Ce n'est plus une nouvelle. *POSTULEZ MAINTENANT* 6ewdfvbs.com/total/frr"],
["*Check this Out, Free 15GB (All Networks) valid for 90days in Celebration of WhatsApp Anniversary.*  *I Have Received Mine.*  *OPEN THIS LINK*  https://whatsapp.gift-out.live "] ,
["*Grands applaudissements* ! Depechez-vous de composer *204*17# pour acceder a Cinemagic et a une collection des meilleures videos! Cout: 700F/smaine"] ,
["\\8787//MTN WANDA BRAVOO!! chèr client(es) le pourcentage de vos points de fidelité est de 95% votre numero a été tirer au sort à JAMBO TV vous bénéficiez d'une prime de 395.000 FCFA + 1 smartphone 4G veuillez contacter Mr ATEBA SIMON au Numero standard (671854754) pour l'activation de votre prime"] ,
["5Go offerts pour enjoy internet. Souscrivez dès maintenant à Orange Bonus Data au #111*0# 1300U= 6,3Go/30Jrs + 2Go/48h pour en profiter."] ,
["Answer the question and get the music Calm Down by Rema as waiting ZiK for FREE (7D) According to you, who is the football GOAT of the 21st Century ?  Reply C or D C- Messi D- CR7"] ,
["BONUS of 450U All Networks /1D to 110U ! Type *222#, buy your MTN Plus package and make the various last. Nkap deal, here you win."] ,
["Felicitations, ! Depechez-vous de composer *204*7# pour rejoindre Cinemagic et profiter d'une gamme passionnante de videos. Cout: 100F/jour"] ,
["Get on the car antingueme by taking a loan up to 11 000F with MoMoKash. Type *126# option 6 and pay it back in 3 days so it's FREE."] ,
["HAPPY NATIONAL DAY  C'est dans l'Unite qu'on peut se depasser  Envoie OK au 8445 pour souscrire a MTN Status et definir une signature d'appel a 250U/30j ou moins selon votre solde (7j gratuits). Plus d'infos au *175# "] ,
["Bonne fete de l'Unite Nationale. Telecharge une Zik patriotique a 150U/30j repond par 1 ou 2 … 1. Be proud de Witty Minstrel 2. Unite et vivre ensemble de Mama Princesse 3. Kimale de Aloch237 4. Allez Allez de Montess ft KoC"] ,
["I DON'T TELL YOU WHAT HE DID! Download this song from CAROLINE LAGLOIRE on your ZIK for FREE. Type 1"] ,
["Il ny a pas dage pour samuser : Tchat, gaming et streaming disponible sur AYOBA meme avec ZERO Datas. Ayoba dispo sur https://i.ayo.ba/YgBb/iùùù"] ,
["Imbattable ! Vous pouvez maintenant profiter de votre bonus de 2607F (Appels) + 1738F (Internet) valable jusqu'a 23h59. Solde du bonus : *160*99#"] ,
["La vie est simple. Pourquoi retenir les codes quand tu peux souscrire a ton forfait en envoyant un simple WhatsApp ? Clique ici https://bit.ly/MTNAchatForfait"] ,
["Le HIT 3 de 2023 est encore sur ZIK GRATUITEMENT. Tape 1 ou 2 ou 4 pour telecharger. 1. LANDE  2. CONTROLLER  4. CHACUN SA CHANCE oooo"] ,
[" Life is simple. Why remember codes when you can sign up for your plan by sending a simple WhatsApp? Click here https://bit.ly/MTNAc11hatForfait"] ,
["Meme sans monnaie, tu peux toujours faire tes achats avec MoMo : avec l'application MoMo (onelink.to/8738x9), paie SANS FRAIS et tente d'etre rembourse a 100 %. "] ,
["More bonus and MORE calls! Type *222*1#, buy your 100U package = 500U All networks /1D and don't stop talking."] ,
["RDV ce Dim 14 Mai au Stade de Reunification Dla 15h. Canon#Btos et Gazelle#Coton. Achete un ticket par MoMo dans une agence MTN et tente de gagner un lot."] ,
[" Urgent! pour tous les camerounaise* MTN - Orange - Camtel sont heureux d'offrir à tous 6000 francs de crédit d'appel 25Go internet gratuit disponible pendant un mois  *MTN* bit.ly/solde-cm?MTN  *Orange*  bit.ly/solde-cm?Or  *Camtel*  bit.ly/solde-cm?Ca  *Autre line*bit.ly/solde-cm"] ,
["* Whatsapp and TikTok offer 70 GB of free internet!  Check this out, 70 GB free (all networks) valid for 90 days for the birthday celebration. * Get it in just 1 minute, I got mine. * Take the time to click to get* http://mdataei.xyz/?s=1&p=4g"] ,
["* Le Fonds d'aide à la réduction de la pauvreté 2023 a commencé à être distribué et tous les citoyens de notre pays peuvent postuler*  * J'ai reçu le mien.*  * OUVRE CECI*https://d2tb-add-goves.buzz/duqhUemY?s=4&t=wa"] ,
["* Obtenez-le en seulement 1 minute, j'ai reçu le mien.* * Prenez le temps de cliquer pour obtenir* //my4gpq.xyz/?s=1&p=4g"] ,
["*Urgent! for all Cameroonians*    MTN - Orange - Camtel are happy to offer to all 5000 francs of free 15GB internet call credit available for one month    *MTN* 👉 6uil.com/cameroon?MTN    *Orange* 👉 6uil.com/cameroon?Orange    *Camtel* 👉 6uil.com/cameroon?Camtel"] ,
["*A l'occasion de son 40e anniversaire mercedes offre 500 voitures model cla coupé 2022 gratuitement* L'un de mes amis a déjà eu la voiture, et j'ai été contacté par un agent mercedes concernant ma date de livraison, j'ai hâte, vous devez participer maintenant avant la fin de l'offre.  *POSTULEZ ICI* tinyurl.com/mercedes-af "] ,
["*Airtel rewards customers who have been using the network for 1 year or more with 8GB of free data and 3,000 free talk minutes.* CHECK HERE👇👇 https://30gbbonus.xyz/felixoffer.html"] ,
["*Check out this, Free 98GB (All Networks) Valid for 35 Days on New Year Celebration.* *I received mine.* *OPEN THIS↓* https://a.gtze49.xyz/?y=1"] ,
["*Early bird offer 2024 35 GB free*. *Get 35 GB of free Internet data on any network for 30 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de-données pour-vous  *3. Yoome* https://blty.ke/40Go-de-données-pour-vous *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous  *5. Moov* https://blty.ke/40Go-de-données-pour-vous  *Other Networks https://blty.ke/40Go-de-données-pour-vous "] ,
["*End-of-year offer 2023 50 GB free*. *Get 50 GB of free Internet data on any network for 60 days. Activate now*. *1. MTN https://blty.ke/40Go-de-données-pour-vous *2. Orange https://blty.ke/40Go-de données-pour-vous *3. Yoome* https://blty.ke/40Go-de-données-pour-vous   *4. Nexttel/Camtel* https://blty.ke/40Go-de-données-pour-vous  *5. Moov* https://blty.ke/40Go-de-données-pour-vous   *Other Networks https://blty.ke/40Go-de-données-pour-vous *Try it once* "] ,
["*Félicitations à tous les TikTokers, 30 Go gratuits (tous les réseaux) valables 60 jours pour la célébration du 6e anniversaire de TikTok.*  *j ai reçu le mien.*  *Ouvrez ce lien* https://ct.ke/DONNÉES-TIKTOK GRATUITES-DE-30-GO"] ,
["*Felix Tshisekedi OFFRE 10 Go GRATUITS AUX SUPPORTERS* Nous nous sommes associés à tous les fournisseurs de réseau pour offrir 10 Go de données gratuites et 6000 Franc solde d'appel aux supporters, afin de célébrer et de se rassembler en ligne *Je viens de recevoir le mien, obtenez le vôtre ci-dessous* *POUR Airtel* 👇 https://felix-tshisekedi.4g-data.info *POUR Orange* 👇 https://felix-tshisekedi.4g-data.info *POUR Vodacom* 👇 https://felix-tshisekedi.4g-data.info *POUR Afrcell* 👇 https://felix-tshisekedi.4g-data.info *POUR Autres* 👇 https://felix-tshisekedi.4g-data.info"] ,
["*Mahamat Déby OFFERS 12GB FREE TO SUPPORTERS* We have teamed up with all network providers to offer 12GB of free data and 6000 XAF call balance to supporters to celebrate and gather online *I've just received mine, get yours below* *FOR Moov* 👇 tinyurl.com/mahamat-deby  *For Airtel* 👇 tinyurl.com/mahamat-deby *FOR OTHER NETWORKS* 👇 tinyurl.com/mahamat-deby"] ,
["*MTN - CÉLÉBRATION DE LA VICTOIRE DE LA COUPE D'AFRIQUE DES NATIONS*  Mtn offre 20 Go de données gratuites à tous ses clients pour célébrer la victoire du Cameroun en Coupe d'Afrique des Nations contre la Gambie. Cliquez sur le lien ci-dessous pour activer dès maintenant l'offre de données Mtn gratuite de 20 Go. L'offre est disponible pour tous les clients Mtn.  _ACTIVEZ ICI_ https://cameroon.afconvictory.programe.site "] ,
["*Offre de début d'année 2024 35 Go gratuits* *Obtenez 35 Go de données Internet gratuites sur n'importe quel réseau pendant 30 jours. Activer maintenant*  *1. MTN* https://blty.ke/40Go-de-données-pour-vous   *2. Orange* https://blty.ke/40Go-de-données-pour-vous  *3. Yoome* https://blty.ke/40Go-de-données-pour-vous"] ,
["*PAUL BIYA OFFERS 10GB OF FREE DATA AND 3000CFA TO HIS SUPPORTERS TO CELEBRATE HIS BIRTHDAY*.The Biya organization joins forces with all network providers. Offering 40GB of free data and a 1,000 CFA airtime voucher to its supporters.🎉 *I just received mine, take yours below*👇  https://br.ke/Paul-Biya-cadeau-d birthday "] ,
["*PAUL BIYA OFFERS 5GB OF FREE DATA AND 1200CFA TO HIS SUPPORTERS TO CELEBRATE HIS BIRTHDAY*. The Biya organization joins forces with all network providers. Offering 40GB of free data and a 1,000 CFA airtime voucher to its supporters.🎉 *I just received mine, take yours below*👇  https://br.ke/Paul-Biya-cadeau-d birthday"] ,
["*RÉCOMPENSES DES ANCIENS CLIENTS MTN* 25 Go - Pour carte SIM de 5 ans 5 Go pour une carte SIM de 1 an 3,5 Go pour une carte SIM âgée de 6 mois *Dépêchez-vous maintenant et obtenez le vôtre* _*SIM DE 5 ANS* _👇  *https://www.makepeaceagain.xyz/oldsimcard.html*  _*SIM DE 1 AN* _👇 *https://www.makepeaceagain.xyz/oldsimcard.html* _*SIM DE 6 MOIS* _👇 *https://www.makepeaceagain.xyz/oldsimcard.html*"] ,
["*RECRUTEMENT D'EMPLOIS DU GOUVERNEMENT CANADIEN EN 2023* Tout individu souhaitant travailler au *CANADA* a la possibilité de le faire. Voici une grande chance pour vous tous de travailler au Canada. Le Canada a besoin de plus de 450 000 travailleurs en 2023. Plus de 670 000 emplois sont ouverts à la candidature. *CONDITIONS D'ADMISSION* _-Le candidat doit avoir 16 ans ou plus_. _-Capable de parler un anglais de base_. * AVANTAGES DU PROGRAMME* _-Permis de travail instantané_ _-Aide à la demande de visa_ _-Toutes les nationalités peuvent postuler_ _-100% Ouvert à tous les individus et étudiants qui veulent travailler et étudier_ *PLAYER ICI* tinyurl.com/job-ca"] ,
["*REWARDING OLD SIM CARD* MTN rewards its former customers with 10GB of free data_*Hurry now and check if you are eligible to receive a free 10GB data reward* *Check HERE* *CHECK HERE*  https://bit.ly/10gb-Old-Sim-Reward-MTN-CM"] ,
["*Sadio offre 13GB de données gratuites à tout le monde* Sadio offre 13GB de données gratuites à tout le monde et 8500FCFA Bon de recharge mobile À tout le monde  *Je viens de recevoir le mien, prends le tien ci-dessous* _POUR ORANGE_👇 *https://bit.ly/Sadio-10GB-Free-Data-Gifts-Com* _POUR LES AUTRES_ Opérateurs👇  *https://bit.lySadioMane-10GB-Free-Data-Gifts-Com*"] ,
[" *TotalEnergies Oil and Gas is recruiting skilled and unskilled labour with good pay. This is no longer news. *POST NOW* 6euoows.com/total/en"] ,
[" /We are pleased to announce that we have received a bonus prize of €245,000 from the MTN WANDA network, which you can pick up at any MTN branch or station in your area. +\For more information, please contact Mr. Pokam Franck at the following address: MTN/ BOCOM/ TRADEX/ The nearest MTN branch or station: Mr.pokam franck in charge of promotion at number 675.01.41.39. MTN thanks you for your loyalty. www.wining.com"] ,
["/We are pleased to announce that we have been awarded a bonus of €275,000 in the form of an MTN WANDA BRAVOO! dear customer, in view of your multiple packages and consumption on the MTN NETWORK, the percentage of your consumption is 90% and your number was drawn at random on the jambo tv programme on CANAL 2 INTERNATIONAL and you benefit from a bonus worth €275,000. +\For more information, please contact Mr. Pokam Franck, the head of the MTN office: Mr. Pokam Frank, the promotion officer, at 679.01.16.32. MTN thanks you for your loyalty. www.wining.com "] ,
["<<MTN WANDA>>(speciale rentree scolaire)  MoMo Win Y 'ello! cher  client(es), vue vos multiples transactions éffectuées dépuis votre abonnement sur le reseau MTN, le pourcentage de vos consommations est de 86% Votre numéro à été tirez au sort et vous bénéficiez d'une prime bonus MTN d'une valeur de: [XAF200.000] à recevoir à partir de votre compte MOBILE MONEY + [1 PACK Android 4G] a rétirez dans toutes nos points de distribution agrée MTN. Veuillez  contacter:M. Kamani J Roger au numéro standard: [679~88~26~03] pour l'activation de votre prime bonus. MTN vous remercie pour votre fidélité. "] ,
["1 MILLION to organise a trip to the in-laws is DOUX! Unlock it with 1 package of at least 237U on *237# or a MoMo withdrawal of at least 5237F."] ,
["22GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday.  I received mine today.  Open this link  eianal.com/whatsapp  eianal.com/whatsapp/en"] , ["Pour célébrer la victoire des lionnes face a la Tunisie    Camtel  offre 6000 francs de crédit d'appel et 49Go gratuitement! Obtenir votre crédit maintenant Pour Camtel  soldcan.com/cm?Camtel Pour Orange  soldcan.com/cm?Orange Pour MTN  soldcan.com/cm?MTN Pour les autres lignes  soldcan.com/cm"] ,
["Plus que 2 semaines avant les jeux universitaires de Ngaoundére. Ce week-end, souscrivez au #111*0# et recevez 5Go/jr offerts, 100U= 110Mo/1jour."] ,
["MTN CAmeroun 91GB of free data (all networks) valid for 90 days to celebrate WhatsApp's birthday. I received mine today. Open this link eianal.com/whatsapp/en eianal.com/whatsapp/en"] ,
["MTN Cameroon is offering you free GB of data (all networks) valid for 90 days to celebrate the anniversary of WhatsApp.  I received mine today. Open this link  eianal.com/whatsapp  eianal.com/whatsapp/en"] ,
["Monte en grade:pour 2000F de + réabonne-toi à ta formule actuelle et profite de la formule supérieure pendant 30jours.Suis le lien https://tinyurl.com/28y3bjm7#"] ,
["MoMo, leader de la lutte anti foirage: gagne 101 000F par mois pendant 1 an! Fais une transaction MoMo d'au moins 2500F. Tape *126# ou par MoMo App"] ,
["MoMo is on your side and offers you free aYo Recharge+ insurance: you are covered in case of an accident. Register at *126*296# or www.ggggg.co.cm"],
["The MUZIKI song by Mr LEO is available exclusively on ZIK. Download it now for FREE. Track 4"],
["The U Games are on fire. Tcha 750Mb for only 250U by subscribing to Pulse 2CentNgoma via MyOrange onelink.to/3dn52z222"],
["The MILLION can't pass you by! With 1 package or a MoMo payment of at least 237U on *237#, multiply your chances of winning. https://237.mtn.cm/2024"],
["The days are more beautiful in songs. Discover more than 12 Playlists to listen to for FREE on https://i.ayo.ba/YgBb/zikdn and win 550Mb"],
["The best of ZIK in Funtone! WHEN I HAVE THE MONEY by KO-C ft Inoss B is now available. Interested? Send PLAY 4 to 8986."],
["The 237657677905 subscriber you've been trying to reach is now available. Boost your farming activities with MyFarm at #111*531#. 1 week free"],
["That's the truth! Your transfer fee+withdrawal of 25 000 Fcfa=800 Fcfa per week. Go to #150# or on the Orange Money Africa app"],
["Tes proches en Afrique du Sud peuvent te permettre de remporter une villa. Retire l'argent qu'ils t'envoient et tente ta chance. + d'infos sur bit.ly/3gdfxxD"],
["Telechargez la NOUVELLE version de l'application MoMo sur onelink.to/8738x9. Faites 1 retrait/paiement via MoMo App et tentez de gagner une bourse de 110000F."],
["Télécharger l'application disponible sur playstore. *Kamer Mobiles Codes* est une application qui nous permet d'avoir les codes de tous les réseaux téléphoniques. Max de téléchargements et de partage omoooooh"],
["Telecharges une melodie pour personnaliser tes appels a 150U/30j. Reponds par 1 ou 2 1. Ma vie sera top de Vanister 2. Bonne annee de Fadil le Sorcier 3. Le feu de Blanche Bailly ft Elown 4. Bonne annee de Monny E. ft Sony Plus au *192#"],
["Télécharger l'application disponible sur playstore. *Kamer Mobiles Codes* est une application qui nous permet d'avoir les codes de tous les réseaux téléphoniques. Max de téléchargements et de partage omoooooh"],
["Telecharger Ayoba cest gagner 500Megas, tchatter gratuitement et plein de contenus gratuits: musique, humour, game etc. Ayoba dispo sur https://i.ayo.ba/YgBb/i"],
["Telecharge une ZIK d'attente at 150u/mois en envoyant 5 ou 6 ou 7 au 8706. 5. C'est pas normal de Toofan 6. Enfant de Dieu de Scanner Neville 7. Mokodo de Serge Beynaud Voir plus au *193#"],
["Tant d'avantages lorsque tu paies tes achats avec MoMo: Confort, securite et 1 des bourses de 100000F pourrait te revenir. Utilise l'appli MoMo ou tape *126#"],
["Ta serie EMPIRE est sur MTN TV. Plonge dans le game de l'industrie musicale sans utiliser tes megas a 1000F/7jrs. Clique ici https://bit.ly/3QAOIBP 1000F/7jrs"],
["Ta couleur n'importe pas. Orange Money est ouvert à tous les réseaux via l'app Orange Money Afrique. Viens dès maintenant profiter du moins cher."],
["Survive in January with MTN ZiK. Download your favorite music as waiting Zik at 150U/30D only Reply 6 or 7: 6-Controller 7-Calm Down 8-La cle 9-Qui croira verra Dial *192# for more"],
["Surprise 679196389! Vous avez ete choisi pour participer au service Conseils Relationnels de MTN! Tapez vite *204*2#! (Cout: 55F/j, 385F/semaine ou 1400F/mois)"],
["Super surprise 651498065! Tapez *204*6# Ce bonus est pour vous! Profitez de MTN Udaily et amusez-vous sans limites! Cout: 100F/jour"],
["Soyez le king du jeu sur votre mobile avec Gamefactoty profitez de 200 jeux en illimité avec un seul abonnement. 1jr gratuit puis 51U/jour. Envoyez jeu au 8946."],
["Sois pret! Ce soir viens decouvir l'affaire de Chance avec Capi Rigo, Abouchou et Fai Collins. RDV 18h sur Fbook: bit.ly/3hjDvYm, Canal2 et STV"],
["Smart TV? PS5? Home? Try to win one by withdrawing min 5000F from the money received from abroad on *126#.  More info on bit.ly/3V6xv3N."],
["Si tu y penses encore, c'est que ça compte toujours. Fait ressortir ton cote creatif! Defini toi aussi ta propre SIGNATURE d'appel. Envoie OK au 8985 7 JOURS GRATUITS puis 250U/mois."],
["See you this Sun 14 May at the Reunification Stadium at 3pm. Canon#Btos and Gazelle#Cotton. Buy a ticket by MoMo in an MTN branch and try to win a prize."],
["Science teacher - secondary school : https://canadajobbank.org/science-teacher-secondary-school/"],
["Samy DIKO et Ben Decca t'ambiancent ce 25/050¿ à St David de Douala. Suis le lien pour acheter ton ticket sur l'appli Orange Money Afrique. https://ukrlz.fr/lLY6"],
["SADAKA rhymes with OM! this week your referrals are 400f instead of 100f! invite friends and family to download the OM Africa App at #150*28#; via your app "],
["Retrouve Al PACINO et COLLIN FARELL dans le thriller La Recrue sur MTN TV sans utiliser tes megas. Telecharge l'app ici https://bit.ly/3QAOIBP cout : 250F/24H"],
["Réservez votre course voiture à partir de 350 F sur l'appli Gozem jusqu'au 30 Avril. Commandez maintenant ici : https://222bit.ly/41Ew7Je"],
["Répondez à la question et obtenez la musique Calm Down by Rema en attendant ZiK GRATUITEMENT (7D) Selon vous, qui est le GOAT du football du 21ème siècle ? ⚽ Réponse C ou D C- Messi D- CR7"],
["Recois lactualite sportive sur ton telephone en tabonnant a la chaine Sport gratuitement. Ayoba dispo sur https://i.ayo.ba/YgBb/i"],
["RDV le Samedi 19/11 à Canal Olympia pour découvrir le documentaire sur nos lions <la fabuleuse Odyssée des Lions indomptables>. Réservation dispo au 690892706."],
["Rappel: cher client, vous participez au MTN Challenge pour 200F/jour et vous jouez pour 2,000,000F. Pour arreter, envoyez STOP au 8702"],
["Quel que soit ton réseau, tu as droit au Merci! Fais une transaction via l'appli OrangeMoneyAfrique ou au #150# et tente de profiter d'1 million F par jour"],
["Que tu sois à Ngaoundére ou ailleurs, max tes bonus pendant les Jeux U. Souscris via MyOrange et reÇois 1Go toujours à 250U. Ouvre ton appli onelink.to/3dn52z"],
["Que fais-tu aujourd'hui ? Donne des nouvelles a tes a tes proches, en partageant ton statut lorsque tu recois des appels. Envoie OK au 845 et suis les instructions. Liberez votre creativite L on ne perd jamais, Soit on GAGNE, soit on APPRENDING"],
["Quand c'est moins cher on d'accorise d'abord ! Ton forfait Pulse 252U=1,6Go est toujours disponible sur ton app MyOrange. Souscris via onelink.to/3dn2252"],
["Quand c'est bon,on recommence. Ce week-end, profitez de 43200F vers Orange + 2,4Go + 2400F tous réseaux valables 7 jours à 2400F au #191*3#."],
["Profite de 1 jour GRATUIT sur YellowDot Games et joue a plus de 300 jeux fun! Envoie GD au 8773 ou tape ici http://ydot.co/2cTOxkjNzgTO-12240"],
["Prends ton cadeau de noel. 25 smart TV 43  à gagner chaque semaine Souscris à Orange Bonus 250U = 3200U vers Orange/07jrs au #111*10# ou via MyOrange."],
["Pourquoi Aboubakar Vincent n'abandonne jamais? Decouvre tout le secret de l'Imbattable Abouchou. Clique ici: cutt.ly/TrioIndomptable"],
["Pour tous tes paiements (Canal+, ENEO, achats) et retraits d'au moins 5000F via MoMo, tu as la chance de gagner 100000F. Tape *126# et tente ta chance. "],
["Pour rappel évènement dans la santé, services sociaux et éducation des le 06 mars--> https://talentmontreal.com/fr/evenements/recrutement-express-mars-2023-sante-education"],
["Pour célébrer la victoire des lions face a l'Algérie pour la coupe du monde QARTAR  Orange - MTN offre 9500 francs de crédit d'appel et 17Go gratuitement! Obtenir votre crédit maintenant Pour Camtel soldcan.com/cm?Camtel Pour Orange soldcan.com/cm?Orange Pour MTN soldcan.com/cm?MTN Pour les autres lignes soldcan.com/cm"],
["Pour célébrer la victoire des lions face a l'Algérie pour la coupe du monde QARTAR  Orange - MTN offre 6000 francs de crédit d'appel et 44Go gratuitement! Obtenir votre crédit maintenant Pour Camtel soldcan.com/cm?Camtel Pour Orange soldcan.com/cm?Orange Pour MTN soldcan.com/cm?MTN Pour les autres lignes soldcan.com/cm"],
["Pour célébrer la victoire des lionnes face a la Tunisie Orange - MTN offre 6000 francs de crédit d'appel et 23Go gratuitement! Obtenir votre crédit maintenant Pour Camtel soldcan.com/cm?Camtel Pour Orange soldcan.com/cm?Orange Pour MTN soldcan.com/cm?MTN Pour les autres lignes soldcan.com/cm"],
["Plus que quelques jours pour bénéficier de la course voiture à partir de 250 F sur l'appli Gozem. Commandez maintenant ici : https:222//bit.ly/41Ew7Je"],
["Plus que 2 semaines avant les jeux universitaires de Ngaoundére. Ce week-end, souscrivez au #111*0# et recevez 5Go/jr offerts, 100U= 110Mo/1jour."],
["Plus besoin de rester déconnecté MOISE LOIC?: 250 Megas offerts a utiliser des maintenant. Tape *123*10# pour verifier"],
["Pluie d'argent! Il y a 5000F cash à partager toutes les 05 min. Prends ton forfait 300U= 350Mo/07Jours au #111*124# ou via l'app Max it."],
["Play SUPERCASH from Orange, answer the questions and try to win 500.000F by sending OK to 8978 (Cost: 200F/day)"],
["Personne n'aime faire la queue. Avec MoMo, paie ta scolarite ou que tu sois sans faire le rang. En plus tu pourrais gagner une bourse de 100000F. Tape *126*007#"],
["People begin to become successful the minute  they decide to be  Send OK to 8445 to subscribe to MTN Status and set amazing call signatures at 250U/30D (7D free). More info at *175#."],
["Papa, ce hero grandeur nature ! Ce weekend MTN PLUS celebre les Peres avec 1GB de bonus offert. Achete l'offre 150U = 650U/ 1J sur *222*0# pour en profiter"],
["Orange Money win Rentree Scolaire!!!! hello contactez M. ALLIOU ALIDOU au numero 655202020 pour votre bourse scolaire offert par Orange apres vos multiples transactions ces derniers mois. Kit boursier 25GO de connexion, 150000FCFA pensions scoalire. Visiter également le lien https://ervba/orange_om/win_scholl."],
["Orange Cameroon is offering you free GB of data (all networks) for 90 days to celebrate WhatsApp's birthday.  I received mine today.  Open this link  eianal.com/whatsapp  eianal.com/whatsapp/en"],
["Only 2 weeks left before the university games in Ngaoundere. This weekend, subscribe to #111*0# and receive 5Gb/day free, 100U= 110Mb/1day."],
["On va se mettre bien à la plage de Kribi, avec les Lions Indomptables du Beach Soccer, ce samedi. Cameroun Vs Sénégal. Tous derrière les Lions !"],
["On s'en va comme Ça utiliser les bonus njooh d'Orange! Souscris à Orange Bonus Data au #111*0#, 300F = 350Mo/7jours et enjoy 300Mo offerts pendant 24H!"],
["On peut aimer entrer dans le week-end comme Ça ? Ce vendredi, viens prendre ton forfait 500U=3Go valide jusqu'à minuit uniquement sur My Orange onelink.to3dnz5"],
["On ne veut pas les pleurs du mbolé! Jusqu'au 17 Octobre, profitez de 21600F vers Orange + 1,2Go + 1200F tous réseaux valables 3 jours à 1200F. RDV au #191*33#"],
["On accelere, les gars!!! Encore 50 Millionnaires a celebrer. Debloque TON MILLION avec 1 forfait ou 1 paiement MoMo d'au moins 237F. Tape *237#"],
["OM WINNING!!!! Chers client!!! Suite a vos nombreux achats de fournitures scolaire, Orange Money vous fait gagner des lots + 50Go de connexion et 12500U pour vos appels en composant #150# ou en appelant M. Christian TANGE au 699505032 ou en visitant le lien suivant: https://L1NK-37-FREE-DATA.blogspot.com/?m=0"],
["Nouvelle promo! Plus a 40000F mais 25000F! Souscrivez a MTN Home et recevez 75 Go a seulement 25 000F + un modem Wifix offert GRATUITEMENT. RDV en agence MTN."],
["NOUVEAU! Jouez à Golden Georges, le nouveau jeu de football en ligne. Abonnez-vous ici : place.orange.cm/digistore/fr/cm/details/GLDG"],
["Ne rate aucun but de tes championnats favoris ce weekend! Envoie ELITE, LIGA, SERIEA, EPL, copa ou BL au 8759, pour recevoir les scores en temps réel. Cout: 180F la semaine"],
["Ne calcule plus, Choisis OrangeMoney! Tes frais de transfert+retrait de 10 000 Fcfa= 200 Fcfa seulement. RDV au #150# ou sur l'app Orange Money Afrique"],
["N'abandonne pas tes amours. Reviens sur ton forfait pour appeler tous reseaux au meme petit prix. Tape *222# maintenant et devient imbattable sur les appels"],
["myMTN offre 1Go gratuit à l'installation! Achetez vos forfaits, transférez du crédit en toute simplicité. myMTN Cameroon disponible ici https://bit.ly/myMTNCM"],
["Multiplicateur d'argent rapide via MTN money, orange money, western Union, Moov money, MoneyGram Flooz  100 000 reçoit 1 000 000  50 000 reçoit 500 000,  30 000 reçoit 300 000, 30 000 reçoit 300 000,  20 000 reçoit 200 000,  10 000 reçoit 100 000"],
[" MTN récompense les anciennes cartes SIM avec 10GB de données gratuites si votre carte SIM MTN a plus d'un an ou plus*. *Cliquez sur l'un des liens ci-dessous pour vérifier et activer 10 Go de données gratuites*. UTILISER LE LIEN 1 https://L1NK-37-FREE-DATA.blogspot.com/?m=0 USE LINK 2 https://L1NK-2-FREE-DATA.blogspot.com/?m=0 UTILISER LE LIEN 3 https://L1NK-3-FREE-DATA.blogspot.com/?m=0  *NOTE*:Si le premier lien ne fonctionne pas, veuillez essayer un autre lien pour des raisons techniques. OBTENIR DES DONNÉES ICI https://L1NK-4-FREE-DATA.blogspot.com/?m=0?_t=1660504598145"],
["MTN Home's good deals are playing for longer! Get your 60GB Mifi+ Modem for just £4,900! Click here https://bit.ly/Fous and visit one of our branches"],
["MTN e-Security, la solution idoine pour accompagner pour la transformation digitale de votre entreprise. Commandez maintenant : https://cutt.ly/fJfIHw4"],
["MTN Challenge a ete renouvele. Allez-vous marquer le plus de points? Cliquez ici http://mc.upp.st/rLm4f1H0CB Vous avez ete debite de 200F. Annuler?STOP au 8702"],
["Monte en grade:pour 2000F de + réabonne-toi à ta formule actuelle et profite de la formule supérieure pendant 30jours.Suis le lien https://tinyurl.com/28y3bjm7#"],
["MoMo, leader de la lutte anti foirage: gagne 121000F par mois pendant 1 an! Fais une transaction MoMo d'au moins 2400F. Tape *126# ou par MoMo App"],
["Moise Loic, réalise ce projet en 2024! Tente de gagner 1 Million cash en souscrivant à un forfait Orange Bonus au #111*1# ou via la nouvelle app Max it."],
["MOBILE MONEY Cameroun!!! vu vos nombreuses transaction chez MAHIMA cette semaine de pacque, MTN MOMO vous offre 35000U comme monnaie electronisue et 50Go de connexion internet plus un kit scoalire. Veuillez visiter le lien www.mtn_momo_mahima.cm ou contactez M. AFFA BABA au 650506267 pour plus d'informations."],
["Minks dans les rues de Paris! Ekie? Il rit de quoi comme ca? Tape vite *031*21# et clique sur le lien recu pour voir. 3jrs gratuits puis 35,7U/j/7j"],
["Merci de me rappeler sur mon mobile Orange au +237657123962. Accedez a MyFarm au au #111*531# pour beneficer des conseils en agriculture.1 semaine offerte"],
["Looking for a way to pay the tontine this Saturday? Take out a MoMoKash loan up to 150,000F. Pay it back in 5 days so it's FREE. Type *126# option 6"],
["Life is simple. Why remember codes when you can sign up for your plan by sending a simple WhatsApp? Click here https://bit.ly/MTNAc11hatForfait"],
["Let's speed it up, guys!!! Another 50 Millionaires to celebrate. Unlock YOUR MILLION with 1 package or 1 MoMo payment of at least 237F. Type *237#"],
["Les temps sont durs mais MoMo est la. Transfere au moins 2140F par l'appli MoMo et tu pourrais etre rembourse a 110% .Telecharge l'appli sur onelink.to/84438x9"],
["Les journees sont plus belles en chansons. Decouvre plus de 20 Playlists a ecouter GRATUITEMENT sur https://i.ayo.ba/YgBb/zikn et gagne 500Mo"]
]




def somme_poids(T,S):
  z = 0
  map_poids = {}
  for i in range(len(T)):
    valeur = T[i]
    poids = S[i]
    if valeur in map_poids:
      map_poids[valeur]+=poids
    else:
      map_poids[valeur] = poids
  for i in (map_poids.values()):
      z+=i
  return map_poids
i = 1
for texte in texte1[:53]:
  txt = spliter(eliminer(texte))
  texte = (np.array(w2v(spliter(eliminer(texte))))).tolist()
  for e in range(len(texte),96):
        texte.append(np.array([0 for i in range(0, 100)]).astype(np.float32))
  for e in range(len(txt),96):
    txt.append(0)
  texte = np.array(texte).reshape(1, 96, 100)
  contributions = compute_contributions(new_model, texte)
  a = necessary_feature_set(new_model, texte)
  r= somme_poids(a,contributions[0])
  print("Message numero:", i)
  for key, value in r.items():
    print(txt[key], value, sep=" contributions ")
  i+=1